In [ ]:
from importlib import reload
#reload(Utilities)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re
from pathlib import Path
import json
import pickle

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version
import copy

import itertools

import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from MECPODf import MECPODf
from MECPOAn import MECPOAn
from MECPOCollection import MECPOCollection
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_General
import Plot_Box_sns
import Plot_Hist
import Plot_Bar
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
def adjust_all_subset_by_mjr_mnr_cause_args(
    mecpo_colls_w_get_args, 
    new_cpo_df_subset_by_mjr_mnr_cause_args,
    new_max_total_counts_args=None, 
    no_outg_idxs=None,
    adjust_barplot_kwargs=None, 
    inplace=False
):
    r"""
    Adjust all of the cpo_df_subset_by_mjr_mnr_cause_args in mecpo_colls_w_get_args WITH THE EXCEPTION OF
    the elements in no_outg_idxs.
      This is because selecting a outage subset from the no-outage data doesn't make any sense!
    
    mecpo_colls_w_get_args:
      A list of dict objects.
      Each dict should have keys:
        mecpo_coll:
          The MECPOCollection object
        cpo_df_name:
          Name of cpo_df to grab from each MECPOAn in the collection for plotting
        cpo_df_subset_by_mjr_mnr_cause_args:
          Arguments to grab any subset of the data by outage major/minor cause
        max_total_counts_args=None:
          Arguments to further select a subset by the maximum number of total counts.
        mecpo_an_order:
          The order in which the analyses in MECPOAn should be plotted
          
    adjust_barplot_kwargs:
      If one wants to, e.g., change hatches for all, etc.
    """
    #-------------------------
    if not inplace:
        mecpo_colls_w_get_args = copy.deepcopy(mecpo_colls_w_get_args)
    #-------------------------
    if no_outg_idxs is not None:
        if not isinstance(no_outg_idxs, list):
            assert(isinstance(no_outg_idxs, int))
            no_outg_idxs = [no_outg_idxs]
        # Not really any harm in having element in no_outg_idxs >= len(mecpo_colls_w_get_args), as this would
        # as this would be the same functionality as no no_outg member identified in list
        # Assertion here just to make sure user understands what's going on
        for idx in no_outg_idxs:
            assert(idx<len(mecpo_colls_w_get_args))
    #-------------------------
    for idx, coll_w_get_args in enumerate(mecpo_colls_w_get_args):
        if adjust_barplot_kwargs is not None:
            Plot_General.adjust_kwargs(
                coll_w_get_args['mecpo_coll'].barplot_kwargs_shared, 
                adjust_barplot_kwargs, 
                inplace=True
            )
        if no_outg_idxs is not None and idx in no_outg_idxs:
            continue
        coll_w_get_args['cpo_df_subset_by_mjr_mnr_cause_args'] =  new_cpo_df_subset_by_mjr_mnr_cause_args
        coll_w_get_args['max_total_counts_args'] =  new_max_total_counts_args
    #-------------------------
    if inplace:
        return
    return mecpo_colls_w_get_args

In [ ]:
def adjust_cpo_df_names_in_mecpo_colls_w_get_args(
    mecpo_colls_w_get_args, 
    cpo_df_names
):
    r"""
    Adjust the cpo_df_name to be used for each MECPOCollection in mecpo_colls_w_get_args.
    
    cpo_df_names:
        The new cpo_df_names, which can be a string, list of lists, or dict
        string:
            A single cpo_df_name which will be used for all MECPOCollections
        list:
            Each element should be a list of length 2.
            The 0th element should contain the index position of the MECPOCollection in mecpo_colls_w_get_args.
            The 1st element should new cpo_df_name to use.
        dict:
            Keys are the index positions of the MECPOCollections in mecpo_colls_w_get_args.
            Values are the associated new cpo_df_names to use.
    """
    #-------------------------
    assert(Utilities.is_object_one_of_types(cpo_df_names, [str, list, dict]))
    if isinstance(cpo_df_names, dict):
        cpo_df_names_dict = cpo_df_names
    elif isinstance(cpo_df_names, str):
        cpo_df_names_dict = {i:cpo_df_names for i in range(len(mecpo_colls_w_get_args))}
    elif isinstance(cpo_df_names, list):
        assert(Utilities.are_all_list_elements_of_type(cpo_df_names, list) and 
               Utilities.are_list_elements_lengths_homogeneous(cpo_df_names, 2))
        cpo_df_names_dict={}
        for idx,cpo_df_name in cpo_df_names:
            assert(isinstance(idx, int))
            assert(idx not in cpo_df_names_dict.keys())
            cpo_df_names_dict[idx] = cpo_df_name
    else:
        assert(0)
    #-------------------------
    for idx,cpo_df_name in cpo_df_names_dict.items():
        assert(idx < len(mecpo_colls_w_get_args))
        mecpo_colls_w_get_args[idx]['cpo_df_name']=cpo_df_name
    #-------------------------
    return mecpo_colls_w_get_args

In [ ]:
def draw_cpo_dfs_full_vs_direct_3x2(
    fig_num, 
    mecpo_an_list_full, 
    mecpo_an_list_drct, 
    cpo_full_dfs_name,
    cpo_drct_dfs_name=None,
    cpo_df_subset_by_mjr_mnr_cause_args=None, 
    max_total_counts_args=None, 
    subplot_titles=None, 
    reason_order=None, 
    n_reason_to_include=None, 
    is_rcpo=True, 
    replace_xtick_labels_with_ints=False, 
    include_xticklabels_for_all=False, 
    suptitle=None, 
    row_major=False, 
    save_args=dict(
        save_fig=False,
        save_dir='', 
        save_name=''
    ), 
    mecpo_an_list_no_outg=None,
    cpo_no_outg_dfs_name=None, 
    **kwargs
):
    r"""
    Compares full to direct cpo_dfs for six different cases.
    Intended to be used to compare 01_05, 06_10, 11_15, 16_20, 21_25, 26_30
    
    mecpo_an_list_full, mecpo_an_list_drct:
      Should be lists (or possibly dicts with matching keys) EACH CONTAINING SIX MECPOAn objects
      
    cpo_full_dfs_name:
      The name of the cpo_full pd.DataFrames, used to retrieve the correct item from mecpo_an_list_full
    cpo_drct_dfs_name:
      Similar to cpo_full_dfs_name, but for the direct collection.  If left equal to None, it will be set equal
      to cpo_full_dfs_name
      
    cpo_df_subset_by_mjr_mnr_cause_args:
      Allows one to select subsets of the pd.DataFrames by the outage type.
      This should be a dict with arguments appropriate for the MECPOAn.get_cpo_df_subset_by_mjr_mnr_cause
        function (except for the cpo_df_name argument, which will be set to cpo_full_dfs_name/cpo_drct_dfs_name)
        
    max_total_counts_args:
      Allows one to further select a subset by the maximum number of total counts.
      This should be a dict with arguments appropriate for the MECPODf.get_cpo_df_subset_below_max_total_counts
        function (except for the cpo_df argument, which will be set to return_df
      
    subplot_titles:
      A list of titles for each subpot (e.g., ['01-05 Days', '06-10 Days', ... '26-30 Days'])
      If None is given and mecpo_an_list_full,mecpo_an_list_full are dicts, the keys will be used
      
    reason_order:
      The order in which to plot the reasons along the x-axis.
      If set to None, the order will be taken from the first DF in mecpo_an_list_full, sorted by mean value
      
    n_reason_to_include:
      Number of reasons to include in the plot.  A reasonable number here is 10, as there is typically a steep drop off
      after this.
      If set equal to None, all will be included.
      
    is_rcpo:
      Set to True if the DataFrames are Reason Counts Per Outage. 
      Set to False if they are Id (enddeviceeventtypeid) Counts Per Outage.
      Used mainly to determine the size of margins to use, as the Reasons tend to be much longer than IDs
      
    replace_xtick_labels_with_ints:
      If True, the xtick labels will be replaced with integers, and a key will be printed to the right of the figures.
      
    include_xticklabels_for_all:
      By default (i.e., when include_xticklabels_for_all is False), the xtick labels are included only for the bottom
        two plots.
      If include_xticklabels_for_all==True, xtick labels are included for each subplot.
      
    suptitle:
      Overall title for collection of plots
      
    row_major:
      If True, the array of axes, axs, is flattened in row-major order.
      If False, the array of axes, axs, is flattened in column-major order.      
      Default: False, so the as the indices increase, the plots move down the first column before
               entering and traversing the second column.
      
    save_args:
      A dict containing information on whether or not to save figure, and, if so, where to save it.
      It can also simply be set to False
      
    kwargs:
        barplot_kwargs_full
        barplot_kwargs_drct
        common_barplot_kwargs
        common_general_kwargs
        sharex
        sharey
        barplot_kwargs_no_outg
    """
    #-------------------------
    assert(len(mecpo_an_list_full)==len(mecpo_an_list_full)==6)
    if isinstance(mecpo_an_list_full, dict) or isinstance(mecpo_an_list_drct, dict):
        assert(isinstance(mecpo_an_list_full, dict) and isinstance(mecpo_an_list_drct, dict))
        assert(mecpo_an_list_full.keys()==mecpo_an_list_drct.keys())
        tmp_full = []
        tmp_drct = []
        tmp_subplot_titles = []
        for key in mecpo_an_list_full.keys():
            tmp_full.append(mecpo_an_list_full[key])
            tmp_drct.append(mecpo_an_list_drct[key])
            tmp_subplot_titles.append(key)
        if subplot_titles is None:
            subplot_titles = tmp_subplot_titles
    if subplot_titles is None:
        subplot_titles = ['' for x in range(6)]
    assert(len(subplot_titles)==6)
    #--------------------------------------------------
    # Unpack kwargs
    barplot_kwargs_full   = kwargs.get('barplot_kwargs_full', dict(facecolor='red', label="Outages (All Xfmrs)"))
    barplot_kwargs_drct   = kwargs.get('barplot_kwargs_drct', dict(facecolor='green', label="Outages (Xfmr tied to Outg. Location ID)"))
    common_barplot_kwargs = kwargs.get('common_barplot_kwargs', dict(alpha=0.25, fill=True, edgecolor='black', hatch='//'))
    
    barplot_kwargs_no_outg = kwargs.get('barplot_kwargs_no_outg', dict(facecolor='orange', label="No Outages"))
    #----------
    xtick_labelrotation=90
    if is_rcpo:
        xtick_labelsize=15
    else:
        xtick_labelsize=20
    ytick_labelsize=25
    if replace_xtick_labels_with_ints:
        xtick_labelrotation=0
        xtick_labelsize=25
    dflt_common_general_kwargs = dict( 
        n_bars_to_include=n_reason_to_include, 
        keep_edges_opaque=True, 
        include_hatches=False, 
        draw_side_by_side=True, 
        draw_single_idx_full_width=None,     
        run_set_general_plotting_args=True, 
        orient='v',
        draw_legend=True, 
        legend_args=dict(fontsize=15), 
        ylabel_args = dict(ylabel=f'', fontsize=0, x=0.0, y=0.8, ha='left', va='bottom'), 
        xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
        tick_args=[dict(axis='x', labelrotation=xtick_labelrotation, labelsize=xtick_labelsize), 
                   dict(axis='y', labelsize=ytick_labelsize)]
    )
    common_general_kwargs = kwargs.get('common_general_kwargs', dflt_common_general_kwargs)
    #----------
    sharex = kwargs.get('sharex', True)
    sharey = kwargs.get('sharey', True)
    #--------------------------------------------------
    if cpo_drct_dfs_name is None:
        cpo_drct_dfs_name = cpo_full_dfs_name
    #-------------------------    
    barplot_kwargs_full = {**barplot_kwargs_full, **common_barplot_kwargs}
    barplot_kwargs_drct = {**barplot_kwargs_drct, **common_barplot_kwargs}
    barplot_kwargs_no_outg = {**barplot_kwargs_no_outg, **common_barplot_kwargs}
    #-------------------------
    if save_args:
        assert('save_fig' in save_args)
        if save_args['save_fig']:
            assert('save_dir' in save_args and 
                   'save_name' in save_args)
    #----------------------------------------------------------------------
    # BUILD THE COLLECTION OF DATAFRAMES
    #----------------------------------------------------------------------
    cpo_dfs_full = []
    for i in range(len(mecpo_an_list_full)):
        cpo_dfs_full.append(
            mecpo_an_list_full[i].get_cpo_df(
                cpo_df_name=cpo_full_dfs_name, 
                cpo_df_subset_by_mjr_mnr_cause_args=cpo_df_subset_by_mjr_mnr_cause_args, 
                max_total_counts_args=max_total_counts_args
            )
        )
            
    #-----
    cpo_dfs_drct = []
    for i in range(len(mecpo_an_list_drct)):
        cpo_dfs_drct.append(
            mecpo_an_list_drct[i].get_cpo_df(
                cpo_df_name=cpo_drct_dfs_name, 
                cpo_df_subset_by_mjr_mnr_cause_args=cpo_df_subset_by_mjr_mnr_cause_args, 
                max_total_counts_args=max_total_counts_args
            )
        )
            
    #-----
    if mecpo_an_list_no_outg is not None:
        assert(len(mecpo_an_list_no_outg)==6)
        if cpo_no_outg_dfs_name is None:
            cpo_no_outg_dfs_name = cpo_full_dfs_name
        cpo_dfs_no_outg = []
        for i in range(len(mecpo_an_list_no_outg)):
            cpo_dfs_no_outg.append(
                mecpo_an_list_no_outg[i].get_cpo_df(
                    cpo_df_name=cpo_no_outg_dfs_name, 
                    cpo_df_subset_by_mjr_mnr_cause_args=cpo_df_subset_by_mjr_mnr_cause_args, 
                    max_total_counts_args=max_total_counts_args
                )
            )
    #----------------------------------------------------------------------
    n_x=2
    n_y=3
    #-----
    fig, axs = Plot_General.default_subplots(n_x=n_x, n_y=n_y, fig_num=fig_num, sharex=sharex, sharey=sharey, 
                                             return_flattened_axes=True, row_major=row_major)
    #-------------------------
    scale_margin_top = None
    if suptitle:
        scale_margin_top = 0.75
    if not replace_xtick_labels_with_ints:
        right = None # i.e., default
        if is_rcpo:
            bottom = 0.5
        else:
            bottom = 0.15
    else:
        bottom = None #i.e., default
        right = 0.75

    subplots_adjust_args = Plot_General.get_subplots_adjust_args_std_3x2(scale_hspace=0.5, scale_margin_left=0.75, 
                                                                         scale_margin_top=scale_margin_top, 
                                                                         bottom=bottom, right=right)
    fig = Plot_General.adjust_subplots_args(fig, subplots_adjust_args)
    #-------------------------
    # When using sharex=True and sharey=True, the default behavior is:
    #   When subplots have a shared x-axis along a column, only the x tick labels of the bottom subplot are created. 
    #   Similarly, when subplots have a shared y-axis along a row, only the y tick labels of the first column subplot are created.
    #   To turn on all tick labels for x(y), set include_all_x(y)_tick_labels to True
    include_all_x_tick_labels=False # If false, only the x tick labels of the bottom subplot are created
    include_all_y_tick_labels=True  # If false, only the y tick labels of the first column subplot are created
    #-------------------------
    if reason_order is None:
        reason_order = cpo_dfs_full[0].mean().sort_values(ascending=False).index.tolist()
    common_general_kwargs['order'] = reason_order
    #-------------------------
    if replace_xtick_labels_with_ints:
        if n_reason_to_include is not None:
            xtick_elements = reason_order[:n_reason_to_include]
        else:
            xtick_elements = reason_order
        xtick_rename_dict = {xtick_el:i+1 for i,xtick_el in enumerate(xtick_elements)}
    #-------------------------
    # NOTE: xticks = np.arange(len(xtick_rename_dict)) below is to ensure all ticks are drawn,
    #       as sometimes mpl draws less ticks when there are many
    if replace_xtick_labels_with_ints:
        Plot_General.adjust_kwargs(
            general_kwargs=common_general_kwargs, 
            new_values_dict=dict(
                ax_args=dict(
                    xticks = np.arange(len(xtick_rename_dict)),
                    xticklabels=list(xtick_rename_dict.values())
                )
            ), 
            append_to_containers=False,
            inplace=True
        )
        
    #------------------------- ACTUAL PLOTTING!!! -------------------------
    #----------------------------------------------------------------------
    # The bottom two plots should always include the xtick labels
    # However, one only needs to worry about this when sharex=False
    # NOTE: If include_xticklabels_for_all==True, one cannot simply set idxs_to_exclude_xticklabels = []
    #       This would work if sharex==False, but if sharex==True, plt.subplots automatically turns off the
    #       tick labels for all but buttom.  Thus, instead of not turning them off (via, e.g., idxs_to_exclude_xticklabels 
    #       = []), one must actively turn them on as well!
    idxs_to_include_xticklabels = [2, 5]
    if row_major:
        idxs_to_include_xticklabels = [4, 5]
    idxs_to_exclude_xticklabels = [x for x in range(6) if x not in idxs_to_include_xticklabels]
    common_general_kwargs_wo_xticklabels = Plot_General.adjust_kwargs(common_general_kwargs, dict(ax_args=dict(xticklabels=[])))
    if include_xticklabels_for_all:
        idxs_to_exclude_xticklabels = []
    #-------------------------
    for i in range(len(cpo_dfs_full)):
        common_general_kwargs_i = common_general_kwargs
        if(not sharex and 
           i in idxs_to_exclude_xticklabels):
            common_general_kwargs_i = common_general_kwargs_wo_xticklabels
        #----------
        dfs_w_args=[
            (cpo_dfs_full[i], barplot_kwargs_full), 
            (cpo_dfs_drct[i], barplot_kwargs_drct)
        ]
        if mecpo_an_list_no_outg is not None:
            dfs_w_args.append((cpo_dfs_no_outg[i], barplot_kwargs_no_outg))
        
        axs[i] = Plot_Bar.plot_multiple_barplots(
            ax=axs[i], 
            dfs_w_args=dfs_w_args, 
            **common_general_kwargs_i, 
            title_args=dict(label=subplot_titles[i], fontsize=20)
        )
        if include_xticklabels_for_all:
            axs[i].tick_params(axis='x', labelbottom=True)
    #---------------------------------------------------------------------------
    # Make all have same scale
    if not sharey:
        Plot_General.make_all_axes_have_same_ylims(axs)
    #---------------------------------------------------------------------------
    subplot_layout_params = Plot_General.get_subplot_layout_params(fig)
    #-----
    if is_rcpo:
        supxlabel = 'Reason'
    else:
        supxlabel = 'ID'
    fig.supxlabel(supxlabel, fontsize=30, 
                  x=subplot_layout_params['right'], ha='right', 
                  y=0.0, va='bottom')
    fig.supylabel('Normalized Counts', fontsize=30, 
                  x=0.0, ha='left',
                  y=subplot_layout_params['top'], va='top')
    if suptitle:
        fig.suptitle(suptitle, fontsize=50, 
                     x=0.5*(subplot_layout_params['left']+subplot_layout_params['right']), ha='center')

    if include_all_y_tick_labels and sharey:
        for ax_i in axs.flatten():
            ax_i.tick_params(axis='y', labelleft=True)

    if include_all_x_tick_labels and sharex:
        for ax_i in axs.flatten():
            ax_i.tick_params(axis='x', labelbottom=True)

    if replace_xtick_labels_with_ints:
        Plot_General.generate_xtick_labels_legend_textbox(
            fig=fig, 
            xtick_rename_dict=xtick_rename_dict, 
            text_x_pos=1.02*subplot_layout_params['right'], 
            text_y_pos=subplot_layout_params['top'])
    #---------------------------------------------------------------------------
    if save_args and save_args['save_fig']:
        Plot_General.save_fig(fig, save_args['save_dir'], save_args['save_name'])   
    #---------------------------------------------------------------------------
    return fig, axs

In [ ]:
def draw_cpo_dfs_full_vs_direct_3x2_v2(
    fig_num, 
    mecpo_colls_w_get_args, 
    reason_order=None, 
    n_reason_to_include=None, 
    is_rcpo=True, 
    replace_xtick_labels_with_ints=False, 
    include_xticklabels_for_all=False, 
    suptitle=None, 
    row_major=False, 
    save_args=dict(
        save_fig=False,
        save_dir='', 
        save_name=''
    ), 
    **kwargs
):
    r"""
    mecpo_colls_w_get_args:
      A list of dict objects.
      Each dict should have keys:
        mecpo_coll:
          The MECPOCollection object
        cpo_df_name:
          Name of cpo_df to grab from each MECPOAn in the collection for plotting
        cpo_df_subset_by_mjr_mnr_cause_args:
          Arguments to grab any subset of the data by outage major/minor cause.
          This should be a dict with arguments appropriate for the MECPOAn.get_cpo_df_subset_by_mjr_mnr_cause
            function (except for the cpo_df_name argument, which will be set to cpo_df_name)  
        max_total_counts_args=None:
          Arguments to further select a subset by the maximum number of total counts.
          This should be a dict with arguments appropriate for the MECPODf.get_cpo_df_subset_below_max_total_counts
            function (except for the cpo_df argument, which will be set to return_df
        mecpo_an_order:
          The order in which the analyses in MECPOAn should be plotted
          
      !!!!!!!!!!!!!!!!!!!!    
      FOR NOW (to be safe):
      !!!!!!!!!!!!!!!!!!!!
        Each mecpo_coll must have the same keys.
          This will ensure that the intended analyses are being plotted together
          These can also be used as the subplot titles
          
    reason_order:
      The order in which to plot the reasons along the x-axis.
      This should be a list of Reasons or IDs (enddeviceeventtypeid).
      If set to an index idx, the order will be taken from mecpo_colls_w_get_args[idx]['mecpo_coll'] via the
        get_rough_reason_ordering method.
      
    n_reason_to_include:
      Number of reasons to include in the plot.  A reasonable number here is 10, as there is typically a steep drop off
      after this.
      If set equal to None, all will be included.
      
    is_rcpo:
      Set to True if the DataFrames are Reason Counts Per Outage. 
      Set to False if they are Id (enddeviceeventtypeid) Counts Per Outage.
      Used mainly to determine the size of margins to use, as the Reasons tend to be much longer than IDs
      
    replace_xtick_labels_with_ints:
      If True, the xtick labels will be replaced with integers, and a key will be printed to the right of the figures.
      
    include_xticklabels_for_all:
      By default (i.e., when include_xticklabels_for_all is False), the xtick labels are included only for the bottom
        two plots.
      If include_xticklabels_for_all==True, xtick labels are included for each subplot.
      
    suptitle:
      Overall title for collection of plots
      
    row_major:
      If True, the array of axes, axs, is flattened in row-major order.
      If False, the array of axes, axs, is flattened in column-major order.      
      Default: False, so the as the indices increase, the plots move down the first column before
               entering and traversing the second column.
      
    save_args:
      A dict containing information on whether or not to save figure, and, if so, where to save it.
      It can also simply be set to False
      
    kwargs:
        common_barplot_kwargs
        common_general_kwargs
        sharex
        sharey
    """
    #-------------------------    
    # Make sure each element in mecpo_colls_w_get_args has the expected keys
    necessary_mecpo_colls_w_get_args_keys = ['mecpo_coll', 'cpo_df_name','mecpo_an_order']
    other_mecpo_colls_w_get_args_keys = ['cpo_df_subset_by_mjr_mnr_cause_args', 'max_total_counts_args']
    for coll_w_get_args in mecpo_colls_w_get_args:
        assert(len(set(necessary_mecpo_colls_w_get_args_keys).difference(set(coll_w_get_args.keys())))==0)
        for other_key in other_mecpo_colls_w_get_args_keys:
            coll_w_get_args[other_key] = coll_w_get_args.get(other_key, None)
    
    # Plotting 6 subplots (3x2), so each MECPOCollection should have 6 MECPOAn objects
    for mecpo_coll in [x['mecpo_coll'] for x in mecpo_colls_w_get_args]:
        assert(mecpo_coll.n_mecpo_ans==6)

    # (Possibly temporary) Restriction: Each MECPOCollection must have the same keys
    mecpo_an_keys = mecpo_colls_w_get_args[0]['mecpo_coll'].mecpo_an_keys
    assert(len(mecpo_an_keys)==6)
    for mecpo_coll in [x['mecpo_coll'] for x in mecpo_colls_w_get_args]:
        assert(len(set(mecpo_an_keys).symmetric_difference(set(mecpo_coll.mecpo_an_keys)))==0)
    #--------------------------------------------------
    # Unpack kwargs
    common_barplot_kwargs = kwargs.get('common_barplot_kwargs', dict(alpha=0.25, fill=True, edgecolor='black', hatch='//'))
    #----------
    xtick_labelrotation=90
    if is_rcpo:
        xtick_labelsize=15
    else:
        xtick_labelsize=20
    ytick_labelsize=25
    if replace_xtick_labels_with_ints:
        xtick_labelrotation=0
        xtick_labelsize=25
    dflt_common_general_kwargs = dict( 
        n_bars_to_include=n_reason_to_include, 
        keep_edges_opaque=True, 
        include_hatches=False, 
        draw_side_by_side=True, 
        draw_single_idx_full_width=None,     
        run_set_general_plotting_args=True, 
        orient='v',
        draw_legend=True, 
        legend_args=dict(fontsize=15), 
        ylabel_args = dict(ylabel=f'', fontsize=0, x=0.0, y=0.8, ha='left', va='bottom'), 
        xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
        tick_args=[dict(axis='x', labelrotation=xtick_labelrotation, labelsize=xtick_labelsize), 
                   dict(axis='y', labelsize=ytick_labelsize)]
    )
    common_general_kwargs = kwargs.get('common_general_kwargs', dflt_common_general_kwargs)
    #----------
    sharex = kwargs.get('sharex', True)
    sharey = kwargs.get('sharey', True)
    #--------------------------------------------------
    if save_args:
        assert('save_fig' in save_args)
        if save_args['save_fig']:
            assert('save_dir' in save_args and 
                   'save_name' in save_args)        
    #----------------------------------------------------------------------
    n_x=2
    n_y=3
    #-----
    fig, axs = Plot_General.default_subplots(n_x=n_x, n_y=n_y, fig_num=fig_num, sharex=sharex, sharey=sharey, 
                                             return_flattened_axes=True, row_major=row_major)
    #-------------------------
    scale_margin_top = None
    if suptitle:
        scale_margin_top = 0.75
    if not replace_xtick_labels_with_ints:
        right = None # i.e., default
        if is_rcpo:
            bottom = 0.5
        else:
            bottom = 0.15
    else:
        bottom = None #i.e., default
        right = 0.75

    subplots_adjust_args = Plot_General.get_subplots_adjust_args_std_3x2(scale_hspace=0.5, scale_margin_left=0.75, 
                                                                         scale_margin_top=scale_margin_top, 
                                                                         bottom=bottom, right=right)
    fig = Plot_General.adjust_subplots_args(fig, subplots_adjust_args)
    #-------------------------
    # When using sharex=True and sharey=True, the default behavior is:
    #   When subplots have a shared x-axis along a column, only the x tick labels of the bottom subplot are created. 
    #   Similarly, when subplots have a shared y-axis along a row, only the y tick labels of the first column subplot are created.
    #   To turn on all tick labels for x(y), set include_all_x(y)_tick_labels to True
    include_all_x_tick_labels=False # If false, only the x tick labels of the bottom subplot are created
    include_all_y_tick_labels=True  # If false, only the y tick labels of the first column subplot are created
    #-------------------------
    if reason_order is None:
        reason_order = 0
    assert(Utilities.is_object_one_of_types(reason_order, [int, list]))
    if isinstance(reason_order, int):
        reason_order = mecpo_colls_w_get_args[reason_order]['mecpo_coll'].get_rough_reason_ordering(
            cpo_df_name=mecpo_colls_w_get_args[reason_order]['cpo_df_name'], 
            cpo_df_subset_by_mjr_mnr_cause_args=mecpo_colls_w_get_args[reason_order]['cpo_df_subset_by_mjr_mnr_cause_args'],
            max_total_counts_args=mecpo_colls_w_get_args[reason_order]['max_total_counts_args']
        )
    #-----
    common_general_kwargs['order'] = reason_order
    #-------------------------
    if replace_xtick_labels_with_ints:
        if n_reason_to_include is not None:
            xtick_elements = reason_order[:n_reason_to_include]
        else:
            xtick_elements = reason_order
        xtick_rename_dict = {xtick_el:i+1 for i,xtick_el in enumerate(xtick_elements)}
    #-------------------------
    # NOTE: xticks = np.arange(len(xtick_rename_dict)) below is to ensure all ticks are drawn,
    #       as sometimes mpl draws less ticks when there are many    
    if replace_xtick_labels_with_ints:
        Plot_General.adjust_kwargs(
            general_kwargs=common_general_kwargs, 
            new_values_dict=dict(
                ax_args=dict(
                    xticks = np.arange(len(xtick_rename_dict)),
                    xticklabels=list(xtick_rename_dict.values())
                )
            ), 
            append_to_containers=False,
            inplace=True
        )
        
    #------------------------- ACTUAL PLOTTING!!! -------------------------
    #----------------------------------------------------------------------
    # The bottom two plots should always include the xtick labels
    # However, one only needs to worry about this when sharex=False
    # NOTE: If include_xticklabels_for_all==True, one cannot simply set idxs_to_exclude_xticklabels = []
    #       This would work if sharex==False, but if sharex==True, plt.subplots automatically turns off the
    #       tick labels for all but buttom.  Thus, instead of not turning them off (via, e.g., idxs_to_exclude_xticklabels 
    #       = []), one must actively turn them on as well!
    idxs_to_include_xticklabels = [2, 5]
    if row_major:
        idxs_to_include_xticklabels = [4, 5]
    idxs_to_exclude_xticklabels = [x for x in range(6) if x not in idxs_to_include_xticklabels]
    common_general_kwargs_wo_xticklabels = Plot_General.adjust_kwargs(common_general_kwargs, dict(ax_args=dict(xticklabels=[])))
    if include_xticklabels_for_all:
        idxs_to_exclude_xticklabels = []
    #-------------------------
    #--------------------------------------------------    
    # Below, qualitatively 
    #   i represents iteration over the 6 subplots
    #     - Note: There are 6 MECPOAn objects in each coll_w_get_args['mecpo_coll'] in mecpo_colls_w_get_args, 
    #             one for each subplot
    #   j represents iteration over the collections (mecpo_colls_w_get_args) for a given subplot
    #-----
    # First, iterate over the plot number/MECPOAn number
    for i_plot in range(len(mecpo_an_keys)):
        # Use mecpo_an_keys[i_plot] as the subplot title
        subplot_title_i = mecpo_an_keys[i_plot]
        #-----
        # Next, build dfs_w_args_i to be plotted by iterating over the MECPOCollection
        # objects (or, more specifically, the members of mecpo_colls_w_get_args), and grabbing
        # the correct DF from each collection
        dfs_w_args_i = []
        for coll_w_get_args_j in mecpo_colls_w_get_args:
            # Get the key for the correct MECPOAn from which to grab the DF
            mecpo_an_key_ij = coll_w_get_args_j['mecpo_an_order'][i_plot] 
            
            # As well as the cpo_df_name, cpo_df_subset_by_mjr_mnr_cause_args, and max_total_counts_args
            # NOTE: Only single value for these, so no [i_plot] accessor
            cpo_df_name_j = coll_w_get_args_j['cpo_df_name']
            cpo_df_subset_by_mjr_mnr_cause_args_j = coll_w_get_args_j['cpo_df_subset_by_mjr_mnr_cause_args']
            max_total_counts_args_j = coll_w_get_args_j['max_total_counts_args']
            #-----
            # Now, get the correct DF from coll_w_get_args_j['mecpo_coll'] given the known mecpo_an_key, cpo_df_name, 
            # cpo_df_subset_by_mjr_mnr_cause_args, and max_total_counts_args
            df_ij = coll_w_get_args_j['mecpo_coll'].get_cpo_df(
                mecpo_an_key=mecpo_an_key_ij, 
                cpo_df_name=cpo_df_name_j, 
                cpo_df_subset_by_mjr_mnr_cause_args=cpo_df_subset_by_mjr_mnr_cause_args_j, 
                max_total_counts_args=max_total_counts_args_j
            )
            #-----
            # Finally, get the barplot_kwargs_shared from coll_w_get_args_j['mecpo_coll'] and join together
            # with common_barplot_kwargs to get barplot_kwargs_ij
            barplot_kwargs_ij = coll_w_get_args_j['mecpo_coll'].barplot_kwargs_shared
            # Note: Order actually is important here.  If keys are shared, the values for the second are kept
            barplot_kwargs_ij = {**common_barplot_kwargs, **barplot_kwargs_ij}
            #-------------------------
            dfs_w_args_i.append((df_ij, barplot_kwargs_ij))
        # END for coll_w_get_args_j in mecpo_colls_w_get_args
        #-----
        # Now, perform actual plotting for i_plot
        common_general_kwargs_i = common_general_kwargs
        if(not sharex and 
           i_plot in idxs_to_exclude_xticklabels):
            common_general_kwargs_i = common_general_kwargs_wo_xticklabels
        #----------
        axs[i_plot] = Plot_Bar.plot_multiple_barplots(
            ax=axs[i_plot], 
            dfs_w_args=dfs_w_args_i,
            **common_general_kwargs_i, 
            title_args=dict(label=subplot_title_i, fontsize=20)
        )
        if include_xticklabels_for_all:
            axs[i_plot].tick_params(axis='x', labelbottom=True)
    #---------------------------------------------------------------------------
    # Make all have same scale
    if not sharey:
        Plot_General.make_all_axes_have_same_ylims(axs)
    #---------------------------------------------------------------------------
    subplot_layout_params = Plot_General.get_subplot_layout_params(fig)
    #-----
    if is_rcpo:
        supxlabel = 'Reason'
    else:
        supxlabel = 'ID'
    fig.supxlabel(supxlabel, fontsize=30, 
                  x=subplot_layout_params['right'], ha='right', 
                  y=0.0, va='bottom')
    fig.supylabel('Normalized Counts', fontsize=30, 
                  x=0.0, ha='left',
                  y=subplot_layout_params['top'], va='top')
    if suptitle:
        fig.suptitle(suptitle, fontsize=50, 
                     x=0.5*(subplot_layout_params['left']+subplot_layout_params['right']), ha='center')

    if include_all_y_tick_labels and sharey:
        for ax_i in axs.flatten():
            ax_i.tick_params(axis='y', labelleft=True)

    if include_all_x_tick_labels and sharex:
        for ax_i in axs.flatten():
            ax_i.tick_params(axis='x', labelbottom=True)

    if replace_xtick_labels_with_ints:
        Plot_General.generate_xtick_labels_legend_textbox(
            fig=fig, 
            xtick_rename_dict=xtick_rename_dict, 
            text_x_pos=1.02*subplot_layout_params['right'], 
            text_y_pos=subplot_layout_params['top'])
    #---------------------------------------------------------------------------
    if save_args and save_args['save_fig']:
        Plot_General.save_fig(fig, save_args['save_dir'], save_args['save_name'])   
    #---------------------------------------------------------------------------
    return fig, axs

In [ ]:
run_testing_data=False
fig_num=0

run_date = '20221216' # Date of data acquisition

date_0 = '2021-01-01' # Lower limit for end events
date_1 = '2021-12-31' # Upper limit for end events

save_fig = False
# save_dir = r'C:\Users\s346557\Documents\Presentations\OutagePredictions\MeterEvents\Figures_by_outg\PDFs'
# save_ext = 'pdf'
save_dir = r'C:\Users\s346557\Documents\Presentations\OutagePredictions\MeterEvents\Figures_by_xfmr\PNGs'
save_ext = 'png'

# dovs_and_end_events_data_dir = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data'
# if run_testing_data:
#     dovs_and_end_events_data_dir = os.path.join(dovs_and_end_events_data_dir, r'TESTING_DATASETS')

date_pd_subdir = f"{date_0.replace('-','')}_{date_1.replace('-','')}"
dovs_and_end_events_data_dir = os.path.join(Utilities.get_local_data_dir(), r'dovs_and_end_events_data', run_date, date_pd_subdir)
assert(os.path.isdir(dovs_and_end_events_data_dir))

files_dir_outg             = os.path.join(dovs_and_end_events_data_dir, r'EndEvents')
files_dir_outg_prim_strict = os.path.join(dovs_and_end_events_data_dir, r'EndEvents_prim_strict')
files_dir_no_outg = os.path.join(dovs_and_end_events_data_dir, r'EndEvents_NoOutg')

file_path_glob = r'end_events_[0-9]*.csv'
file_path_regex = None

assert_all_cols_equal=True
include_normalize_by_nSNs=True
inclue_zero_counts=True
return_multiindex_outg_reason=False
return_normalized_separately=False


if include_normalize_by_nSNs and not return_normalized_separately:
    normalize_by_nSNs_included=True
else:
    normalize_by_nSNs_included=False
    
xfmr_equip_typ_nms_of_interest = ['TRANSFORMER, OH', 'TRANSFORMER, UG']

normalize_by_time_interval=True

In [ ]:
rcpo_full_dfs_name='rcpo_df_norm_by_xfmr_nSNs'
rcpo_drct_dfs_name='rcpo_df_norm_by_xfmr_nSNs'
rcpo_no_outg_dfs_name='rcpo_df_norm_by_xfmr_nSNs'
rcpo_order_df_name = rcpo_drct_dfs_name #TODO probably change this to index in mecpo_colls_w_get_args instead


icpo_full_dfs_name='icpo_df_norm_by_xfmr_nSNs'
icpo_drct_dfs_name='icpo_df_norm_by_xfmr_nSNs'
icpo_no_outg_dfs_name='icpo_df_norm_by_xfmr_nSNs'

mecpo_idx_for_ordering = 1

In [ ]:
#-------------------------
mecpo_coll_full = MECPOCollection(
    mecpo_coll=None, 
    coll_label = 'Outages (All Xfmrs)', 
    barplot_kwargs_shared = dict(facecolor='red'), 
    read_and_load_all_pickles=True, 
    pkls_base_dir=os.path.join(dovs_and_end_events_data_dir, 'rcpo_dfs_GRP_BY_OUTG_AND_XFMR'),
    days_min_max_outg_td_windows=[
        [1,5], [6,10], [11,15], [16,20], [21,25], [26,30]
    ], 
    pkls_sub_dirs=None,
    naming_tag=None,
    normalize_by_time_interval=normalize_by_time_interval, 
    are_no_outg=False    
)
#-------------------------
mecpo_coll_drct = MECPOCollection(
    mecpo_coll=None, 
    coll_label = 'Outages (Xfmr tied to Outg. Location ID)', 
    barplot_kwargs_shared = dict(facecolor='green'), 
    read_and_load_all_pickles=True, 
    pkls_base_dir=os.path.join(dovs_and_end_events_data_dir, 'rcpo_dfs_GRP_BY_OUTG_AND_XFMR'),
    days_min_max_outg_td_windows=[
        [1,5], [6,10], [11,15], [16,20], [21,25], [26,30]
    ], 
    pkls_sub_dirs=None,
    naming_tag='_prim_strict',
    normalize_by_time_interval=normalize_by_time_interval, 
    are_no_outg=False    
)
#-------------------------
mecpo_coll_no_outg = MECPOCollection(
    mecpo_coll=None, 
    coll_label = 'No Outages', 
    barplot_kwargs_shared = dict(facecolor='orange'), 
    read_and_load_all_pickles=True, 
    pkls_base_dir=os.path.join(dovs_and_end_events_data_dir, 'rcpo_dfs_GRP_BY_NO_OUTG_AND_XFMR'),
    days_min_max_outg_td_windows=[
        [1,5], [6,10], [11,15], [16,20], [21,25], [26,30]
    ], 
    pkls_sub_dirs=None,
    naming_tag='_no_outg',
    normalize_by_time_interval=normalize_by_time_interval, 
    are_no_outg=True    
)
#--------------------------------------------------
old_to_new_keys_dict = {
    'outg_td_window_1_to_5_days'  :'01-05 Days',
    'outg_td_window_6_to_10_days' :'06-10 Days',
    'outg_td_window_11_to_15_days':'11-15 Days',
    'outg_td_window_16_to_20_days':'16-20 Days',
    'outg_td_window_21_to_25_days':'21-25 Days',
    'outg_td_window_26_to_30_days':'26-30 Days'
}
mecpo_coll_full.change_mecpo_an_keys(old_to_new_keys_dict)
mecpo_coll_drct.change_mecpo_an_keys(old_to_new_keys_dict)
mecpo_coll_no_outg.change_mecpo_an_keys(old_to_new_keys_dict)
#--------------------------------------------------
mecpo_an_keys = list(old_to_new_keys_dict.values())

In [ ]:
mecpo_coll_full.make_cpo_columns_equal(drop_empty_cpo_dfs=True)
mecpo_coll_drct.make_cpo_columns_equal(drop_empty_cpo_dfs=True)
mecpo_coll_no_outg.make_cpo_columns_equal(drop_empty_cpo_dfs=True)

MECPOCollection.make_cpo_columns_equal_between_mecpo_colls(
    mecpo_colls = [
        mecpo_coll_full, 
        mecpo_coll_drct, 
        mecpo_coll_no_outg
    ], 
    drop_empty_cpo_dfs=True
)
#-------------------------
if not(rcpo_full_dfs_name==rcpo_drct_dfs_name==rcpo_no_outg_dfs_name):
    MECPOCollection.make_mixed_cpo_columns_equal_between_mecpo_colls(
        mecpo_colls_with_cpo_df_names = [
            [mecpo_coll_full, rcpo_full_dfs_name], 
            [mecpo_coll_drct, rcpo_drct_dfs_name], 
            [mecpo_coll_no_outg, rcpo_no_outg_dfs_name]
        ], 
        segregate_by_mecpo_an_keys=False
    )
#-------------------------
if not(icpo_full_dfs_name==icpo_drct_dfs_name==icpo_no_outg_dfs_name):
    MECPOCollection.make_mixed_cpo_columns_equal_between_mecpo_colls(
        mecpo_colls_with_cpo_df_names = [
            [mecpo_coll_full, icpo_full_dfs_name], 
            [mecpo_coll_drct, icpo_drct_dfs_name], 
            [mecpo_coll_no_outg, icpo_no_outg_dfs_name]
        ], 
        segregate_by_mecpo_an_keys=False
    )

# =================================================

In [ ]:
include_power_down_minus_up=True
#-------------------------
#-------------------------
# Remove all reasons containing 'cleared'
mecpo_coll_full.remove_reasons_from_all_rcpo_dfs(['.*cleared.*', '.*Test Mode.*'])
mecpo_coll_drct.remove_reasons_from_all_rcpo_dfs(['.*cleared.*', '.*Test Mode.*'])
mecpo_coll_no_outg.remove_reasons_from_all_rcpo_dfs(['.*cleared.*', '.*Test Mode.*'])

#-------------------------
# Combine reasons using the standard combine (see dflt_patterns_and_replace in MECPODf.combine_cpo_df_reasons
#   for the list of default patterns_and_replace)
mecpo_coll_full.combine_reasons_in_all_rcpo_dfs(initial_strip=True, initial_punctuation_removal=True)
mecpo_coll_drct.combine_reasons_in_all_rcpo_dfs(initial_strip=True, initial_punctuation_removal=True)
mecpo_coll_no_outg.combine_reasons_in_all_rcpo_dfs(initial_strip=True, initial_punctuation_removal=True) 

#-------------------------
# Build power down minus power up counts
if include_power_down_minus_up:
    mecpo_coll_full.delta_cpo_df_reasons_in_all_rcpo_dfs(
        reasons_1='Primary Power Down occurred for meter',
        reasons_2='Primary Power Up occurred for meter',
        delta_reason_name='Power Down Minus Up'
    )
    mecpo_coll_drct.delta_cpo_df_reasons_in_all_rcpo_dfs(
        reasons_1='Primary Power Down occurred for meter',
        reasons_2='Primary Power Up occurred for meter',
        delta_reason_name='Power Down Minus Up'
    )
    mecpo_coll_no_outg.delta_cpo_df_reasons_in_all_rcpo_dfs(
        reasons_1='Primary Power Down occurred for meter',
        reasons_2='Primary Power Up occurred for meter',
        delta_reason_name='Power Down Minus Up'
    )

In [ ]:
#-------------------------
# Don't want to include SNs or nSNs cols (and similar) in plotting, so remove
mecpo_coll_full.remove_SNs_cols_from_all_cpo_dfs()
mecpo_coll_drct.remove_SNs_cols_from_all_cpo_dfs()
mecpo_coll_no_outg.remove_SNs_cols_from_all_cpo_dfs()

In [ ]:
# To use in old plotting method (although, may want to still keep this around?)
mecpo_an_list_full    = mecpo_coll_full.get_mecpo_ans(order=mecpo_an_keys)
mecpo_an_list_drct    = mecpo_coll_drct.get_mecpo_ans(order=mecpo_an_keys)
mecpo_an_list_no_outg = mecpo_coll_no_outg.get_mecpo_ans(order=mecpo_an_keys)

In [ ]:
mecpo_coll_w_get_args_full = dict(
    mecpo_coll=mecpo_coll_full, 
    cpo_df_name=rcpo_full_dfs_name, 
    cpo_df_subset_by_mjr_mnr_cause_args=None, 
    mecpo_an_order=mecpo_an_keys
)
#-----
mecpo_coll_w_get_args_drct = dict(
    mecpo_coll=mecpo_coll_drct, 
    cpo_df_name=rcpo_drct_dfs_name, 
    cpo_df_subset_by_mjr_mnr_cause_args=None, 
    mecpo_an_order=mecpo_an_keys
)
#-----
mecpo_coll_w_get_args_no_outg = dict(
    mecpo_coll=mecpo_coll_no_outg, 
    cpo_df_name=rcpo_no_outg_dfs_name, 
    cpo_df_subset_by_mjr_mnr_cause_args=None, 
    mecpo_an_order=mecpo_an_keys
)
#-------------------------
mecpo_colls_w_get_args = [
    mecpo_coll_w_get_args_full, 
    mecpo_coll_w_get_args_drct, 
    mecpo_coll_w_get_args_no_outg
]
no_outg_idxs=2

In [ ]:
subset_args_all_outgs = None
subset_args_dl_ol = dict(
    mjr_cause='DL',
    mnr_cause='OL', 
    addtnl_slicers=None, 
)
subset_args_dl_eqf = dict(
    mjr_cause='DL',
    mnr_cause='EQF', 
    addtnl_slicers=None, 
)
subset_args_dl_eqf_xfmr = dict(
    mjr_cause='DL',
    mnr_cause='EQF', 
    addtnl_slicers=[dict(column='EQUIP_TYP_NM', value=['TRANSFORMER, OH', 'TRANSFORMER, UG'])], 
)

In [ ]:
common_barplot_kwargs = dict(alpha=0.25, fill=True, edgecolor='black', hatch='//')
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
reason_order = mecpo_coll_drct.get_cpo_df('01-05 Days', 'rcpo_df_norm').mean().sort_values(ascending=False).index.tolist()
n_reason_to_include=15
ax0 = Plot_Bar.plot_multiple_barplots(
    ax=ax0, 
    dfs_w_args=[
        (mecpo_coll_full.get_cpo_df('01-05 Days', 'rcpo_df_norm'), {**dict(facecolor='red', hatch=None, label="Outages (All Xfmrs)"), **common_barplot_kwargs}), 
        (mecpo_coll_drct.get_cpo_df('01-05 Days', 'rcpo_df_norm'), {**dict(facecolor='green', hatch=None, label="Outages (Xfmr tied to Outg. Location ID)"), **common_barplot_kwargs})
    ], 
    order=reason_order, 
    n_bars_to_include=n_reason_to_include, 
    keep_edges_opaque=True, 
    include_hatches=False, 
    draw_side_by_side=True, 
    draw_single_idx_full_width=None,     
    run_set_general_plotting_args=True, 
    orient='v',
    draw_legend=False, 
    legend_args=dict(fontsize=15), 
    title_args=dict(label=f'Title', fontsize=20), 
    ylabel_args = dict(ylabel=f'y-label', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='x-label', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=90, labelsize=7), 
              dict(axis='y', labelsize=15)]
)
# Plot_General.save_fig(fig, save_dir, f'fig_0.{save_ext}')

In [ ]:
save_name = f'RCPO_dfs_3x2_full_v0.{save_ext}'
#-----
cpo_full_dfs_name='rcpo_df_norm_by_xfmr_nSNs'
cpo_drct_dfs_name='rcpo_df_norm_by_xfmr_nSNs'
cpo_no_outg_dfs_name='rcpo_df_norm_by_xfmr_nSNs'

cpo_df_subset_by_mjr_mnr_cause_args=None

reason_order_df_name = 'rcpo_df_norm_by_xfmr_nSNs'
reason_order=mecpo_coll_drct.get_cpo_df(
    '01-05 Days', 
    reason_order_df_name, 
    cpo_df_subset_by_mjr_mnr_cause_args=cpo_df_subset_by_mjr_mnr_cause_args
).mean().sort_values(ascending=False).index.tolist()
#-----
fig,axs = draw_cpo_dfs_full_vs_direct_3x2(
    fig_num=1000, 
    mecpo_an_list_full = mecpo_an_list_full, 
    mecpo_an_list_drct = mecpo_an_list_drct, 
    cpo_full_dfs_name=cpo_full_dfs_name,
    cpo_drct_dfs_name=cpo_drct_dfs_name,
    cpo_df_subset_by_mjr_mnr_cause_args=cpo_df_subset_by_mjr_mnr_cause_args,
    subplot_titles=[
        '01-05 Days', 
        '06-10 Days', 
        '11-15 Days', 
        '16-20 Days', 
        '21-25 Days', 
        '26-30 Days'
    ], 
    reason_order=reason_order, 
    n_reason_to_include=10, 
    is_rcpo=True, 
    replace_xtick_labels_with_ints=True, 
    include_xticklabels_for_all=True, 
    suptitle='Full', 
    row_major=False, 
    save_args=dict(
        save_fig=save_fig,
        save_dir=save_dir, 
        save_name=save_name
    ), 
    sharex=True,
    sharey=True, 
    mecpo_an_list_no_outg=mecpo_an_list_no_outg, 
    cpo_no_outg_dfs_name=cpo_no_outg_dfs_name
)

In [ ]:
save_name = f'RCPO_dfs_3x2_full.{save_ext}'

mecpo_colls_w_get_args = adjust_all_subset_by_mjr_mnr_cause_args(
    mecpo_colls_w_get_args=mecpo_colls_w_get_args, 
    new_cpo_df_subset_by_mjr_mnr_cause_args=subset_args_all_outgs,
    new_max_total_counts_args=None, 
    no_outg_idxs=no_outg_idxs, 
    adjust_barplot_kwargs=dict(hatch=None), 
    inplace=False
)

reason_order=mecpo_coll_drct.get_cpo_df(
    '01-05 Days', 
    mecpo_colls_w_get_args[mecpo_idx_for_ordering]['cpo_df_name'], 
    cpo_df_subset_by_mjr_mnr_cause_args=mecpo_colls_w_get_args[mecpo_idx_for_ordering]['cpo_df_subset_by_mjr_mnr_cause_args']
).mean().sort_values(ascending=False).index.tolist()

#-----
fig,axs = draw_cpo_dfs_full_vs_direct_3x2_v2(
    fig_num=1000, 
    mecpo_colls_w_get_args=mecpo_colls_w_get_args, 
    reason_order=reason_order, 
    n_reason_to_include=10, 
    is_rcpo=True, 
    replace_xtick_labels_with_ints=True, 
    include_xticklabels_for_all=True, 
    suptitle='Full', 
    row_major=False, 
    save_args=dict(
        save_fig=save_fig,
        save_dir=save_dir, 
        save_name=save_name
    ), 
    sharex=True,
    sharey=True
)

In [ ]:
# NEW!!!!!!!!!!!!!!!!!!!!!!!!
# with max_total_counts
save_name = f'RCPO_dfs_3x2_full_max_counts_150.{save_ext}'

mecpo_colls_w_get_args = adjust_all_subset_by_mjr_mnr_cause_args(
    mecpo_colls_w_get_args=mecpo_colls_w_get_args, 
    new_cpo_df_subset_by_mjr_mnr_cause_args=subset_args_all_outgs,
    new_max_total_counts_args=150, 
    no_outg_idxs=no_outg_idxs, 
    adjust_barplot_kwargs=dict(hatch=None), 
    inplace=False
)

reason_order=mecpo_coll_drct.get_cpo_df(
    '01-05 Days', 
    mecpo_colls_w_get_args[mecpo_idx_for_ordering]['cpo_df_name'], 
    cpo_df_subset_by_mjr_mnr_cause_args=mecpo_colls_w_get_args[mecpo_idx_for_ordering]['cpo_df_subset_by_mjr_mnr_cause_args']
).mean().sort_values(ascending=False).index.tolist()

#-----
fig,axs = draw_cpo_dfs_full_vs_direct_3x2_v2(
    fig_num=1000, 
    mecpo_colls_w_get_args=mecpo_colls_w_get_args, 
    reason_order=reason_order, 
    n_reason_to_include=10, 
    is_rcpo=True, 
    replace_xtick_labels_with_ints=True, 
    include_xticklabels_for_all=True, 
    suptitle='Full', 
    row_major=False, 
    save_args=dict(
        save_fig=save_fig,
        save_dir=save_dir, 
        save_name=save_name
    ), 
    sharex=True,
    sharey=True
)

In [ ]:
save_name = f'RCPO_dfs_3x2_dl_ol.{save_ext}'

mecpo_colls_w_get_args = adjust_all_subset_by_mjr_mnr_cause_args(
    mecpo_colls_w_get_args=mecpo_colls_w_get_args, 
    new_cpo_df_subset_by_mjr_mnr_cause_args=subset_args_dl_ol,
    new_max_total_counts_args=None, 
    no_outg_idxs=no_outg_idxs, 
    adjust_barplot_kwargs=dict(hatch='//'), 
    inplace=False
)

reason_order=mecpo_coll_drct.get_cpo_df(
    '01-05 Days', 
    mecpo_colls_w_get_args[mecpo_idx_for_ordering]['cpo_df_name'], 
    cpo_df_subset_by_mjr_mnr_cause_args=mecpo_colls_w_get_args[mecpo_idx_for_ordering]['cpo_df_subset_by_mjr_mnr_cause_args']
).mean().sort_values(ascending=False).index.tolist()

#-----
fig,axs = draw_cpo_dfs_full_vs_direct_3x2_v2(
    fig_num=1000, 
    mecpo_colls_w_get_args=mecpo_colls_w_get_args, 
    reason_order=reason_order, 
    n_reason_to_include=10, 
    is_rcpo=True, 
    replace_xtick_labels_with_ints=True, 
    suptitle='DL_OL', 
    row_major=False, 
    save_args=dict(
        save_fig=save_fig,
        save_dir=save_dir, 
        save_name=save_name
    ), 
    sharex=True,
    sharey=True
)

In [ ]:
save_name = f'RCPO_dfs_3x2_dl_eqf.{save_ext}'

mecpo_colls_w_get_args = adjust_all_subset_by_mjr_mnr_cause_args(
    mecpo_colls_w_get_args=mecpo_colls_w_get_args, 
    new_cpo_df_subset_by_mjr_mnr_cause_args=subset_args_dl_eqf,
    new_max_total_counts_args=None, 
    no_outg_idxs=no_outg_idxs, 
    adjust_barplot_kwargs=dict(hatch='-'), 
    inplace=False
)

reason_order=mecpo_coll_drct.get_cpo_df(
    '01-05 Days', 
    mecpo_colls_w_get_args[mecpo_idx_for_ordering]['cpo_df_name'], 
    cpo_df_subset_by_mjr_mnr_cause_args=mecpo_colls_w_get_args[mecpo_idx_for_ordering]['cpo_df_subset_by_mjr_mnr_cause_args']
).mean().sort_values(ascending=False).index.tolist()

#-----
fig,axs = draw_cpo_dfs_full_vs_direct_3x2_v2(
    fig_num=1000, 
    mecpo_colls_w_get_args=mecpo_colls_w_get_args, 
    reason_order=reason_order, 
    n_reason_to_include=10, 
    is_rcpo=True, 
    replace_xtick_labels_with_ints=True, 
    suptitle='DL_EQF', 
    row_major=False, 
    save_args=dict(
        save_fig=save_fig,
        save_dir=save_dir, 
        save_name=save_name
    ), 
    sharex=True,
    sharey=True
)

In [ ]:
save_name = f'RCPO_dfs_3x2_dl_eqf_xfmr.{save_ext}'

mecpo_colls_w_get_args = adjust_all_subset_by_mjr_mnr_cause_args(
    mecpo_colls_w_get_args=mecpo_colls_w_get_args, 
    new_cpo_df_subset_by_mjr_mnr_cause_args=subset_args_dl_eqf_xfmr,
    new_max_total_counts_args=None, 
    no_outg_idxs=no_outg_idxs, 
    adjust_barplot_kwargs=dict(hatch='\\\\'), 
    inplace=False
)

reason_order=mecpo_coll_drct.get_cpo_df(
    '01-05 Days', 
    mecpo_colls_w_get_args[mecpo_idx_for_ordering]['cpo_df_name'], 
    cpo_df_subset_by_mjr_mnr_cause_args=mecpo_colls_w_get_args[mecpo_idx_for_ordering]['cpo_df_subset_by_mjr_mnr_cause_args']
).mean().sort_values(ascending=False).index.tolist()

#-----
fig,axs = draw_cpo_dfs_full_vs_direct_3x2_v2(
    fig_num=1000, 
    mecpo_colls_w_get_args=mecpo_colls_w_get_args, 
    reason_order=reason_order,  
    n_reason_to_include=10, 
    is_rcpo=True, 
    replace_xtick_labels_with_ints=True, 
    suptitle='DL_EQF_XFMR', 
    row_major=False, 
    save_args=dict(
        save_fig=save_fig,
        save_dir=save_dir, 
        save_name=save_name
    ), 
    sharex=True,
    sharey=True
)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
ax0 = Plot_Bar.plot_barplot(
    ax=ax0, 
    df = mecpo_coll_no_outg.get_cpo_df('01-05 Days', 'rcpo_df_norm'), 
    order=mecpo_coll_no_outg.get_cpo_df('01-05 Days', 'rcpo_df_norm').mean().sort_values(ascending=False).index.tolist(), 
    n_bars_to_include=10, 
    barplot_kwargs=dict(label='No Outages'), 
    orient='v', 
    draw_legend=True, 
    legend_args=dict(fontsize=15), 
    title_args=dict(label=f'End Events', fontsize=20), 
    ylabel_args = dict(ylabel=f'Normalized Counts', fontsize=20, x=0.0, y=0.4, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='Reason', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=90, labelsize=15), 
              dict(axis='y', labelsize=15)]
)
# Plot_General.save_fig(fig, save_dir, f'fig_7.{save_ext}')

# ICPO DFs

In [ ]:
mecpo_colls_w_get_args = adjust_cpo_df_names_in_mecpo_colls_w_get_args(
    mecpo_colls_w_get_args, 
    cpo_df_names = {
        0:icpo_full_dfs_name, 
        1:icpo_drct_dfs_name, 
        2:icpo_no_outg_dfs_name
    }
)

In [ ]:
save_name = f'ICPO_dfs_3x2_full_best.{save_ext}'

mecpo_colls_w_get_args = adjust_all_subset_by_mjr_mnr_cause_args(
    mecpo_colls_w_get_args=mecpo_colls_w_get_args, 
    new_cpo_df_subset_by_mjr_mnr_cause_args=subset_args_all_outgs,
    new_max_total_counts_args=None, 
    no_outg_idxs=no_outg_idxs, 
    adjust_barplot_kwargs=dict(hatch=None), 
    inplace=False
)

reason_order=mecpo_coll_drct.get_cpo_df(
    '01-05 Days', 
    mecpo_colls_w_get_args[mecpo_idx_for_ordering]['cpo_df_name'], 
    cpo_df_subset_by_mjr_mnr_cause_args=mecpo_colls_w_get_args[mecpo_idx_for_ordering]['cpo_df_subset_by_mjr_mnr_cause_args']
).mean().sort_values(ascending=False).index.tolist()

#-----
fig,axs = draw_cpo_dfs_full_vs_direct_3x2_v2(
    fig_num=1000, 
    mecpo_colls_w_get_args=mecpo_colls_w_get_args, 
    reason_order=reason_order, 
    n_reason_to_include=10, 
    is_rcpo=False, 
    replace_xtick_labels_with_ints=False, 
    include_xticklabels_for_all=False, 
    suptitle='Full', 
    row_major=False, 
    save_args=dict(
        save_fig=save_fig,
        save_dir=save_dir, 
        save_name=save_name
    ), 
    sharex=True,
    sharey=True
)

In [ ]:
save_name = f'ICPO_dfs_3x2_dl_ol_best.{save_ext}'

mecpo_colls_w_get_args = adjust_all_subset_by_mjr_mnr_cause_args(
    mecpo_colls_w_get_args=mecpo_colls_w_get_args, 
    new_cpo_df_subset_by_mjr_mnr_cause_args=subset_args_dl_ol,
    new_max_total_counts_args=None, 
    no_outg_idxs=no_outg_idxs, 
    adjust_barplot_kwargs=dict(hatch=None), 
    inplace=False
)

reason_order=mecpo_coll_drct.get_cpo_df(
    '01-05 Days', 
    mecpo_colls_w_get_args[mecpo_idx_for_ordering]['cpo_df_name'], 
    cpo_df_subset_by_mjr_mnr_cause_args=mecpo_colls_w_get_args[mecpo_idx_for_ordering]['cpo_df_subset_by_mjr_mnr_cause_args']
).mean().sort_values(ascending=False).index.tolist()

#-----
fig,axs = draw_cpo_dfs_full_vs_direct_3x2_v2(
    fig_num=1000, 
    mecpo_colls_w_get_args=mecpo_colls_w_get_args, 
    reason_order=reason_order, 
    n_reason_to_include=10, 
    is_rcpo=False, 
    replace_xtick_labels_with_ints=False, 
    include_xticklabels_for_all=False, 
    suptitle='DL_OL', 
    row_major=False, 
    save_args=dict(
        save_fig=save_fig,
        save_dir=save_dir, 
        save_name=save_name
    ), 
    sharex=True,
    sharey=True
)

In [ ]:
save_name = f'ICPO_dfs_3x2_dl_eqf_best.{save_ext}'

mecpo_colls_w_get_args = adjust_all_subset_by_mjr_mnr_cause_args(
    mecpo_colls_w_get_args=mecpo_colls_w_get_args, 
    new_cpo_df_subset_by_mjr_mnr_cause_args=subset_args_dl_eqf,
    new_max_total_counts_args=None, 
    no_outg_idxs=no_outg_idxs, 
    adjust_barplot_kwargs=dict(hatch=None), 
    inplace=False
)

reason_order=mecpo_coll_drct.get_cpo_df(
    '01-05 Days', 
    mecpo_colls_w_get_args[mecpo_idx_for_ordering]['cpo_df_name'], 
    cpo_df_subset_by_mjr_mnr_cause_args=mecpo_colls_w_get_args[mecpo_idx_for_ordering]['cpo_df_subset_by_mjr_mnr_cause_args']
).mean().sort_values(ascending=False).index.tolist()

#-----
fig,axs = draw_cpo_dfs_full_vs_direct_3x2_v2(
    fig_num=1000, 
    mecpo_colls_w_get_args=mecpo_colls_w_get_args, 
    reason_order=reason_order, 
    n_reason_to_include=10, 
    is_rcpo=False, 
    replace_xtick_labels_with_ints=False, 
    include_xticklabels_for_all=False, 
    suptitle='DL_EQF', 
    row_major=False, 
    save_args=dict(
        save_fig=save_fig,
        save_dir=save_dir, 
        save_name=save_name
    ), 
    sharex=True,
    sharey=True
)

In [ ]:
save_name = f'ICPO_dfs_3x2_dl_eqf_xfmr_best.{save_ext}'

mecpo_colls_w_get_args = adjust_all_subset_by_mjr_mnr_cause_args(
    mecpo_colls_w_get_args=mecpo_colls_w_get_args, 
    new_cpo_df_subset_by_mjr_mnr_cause_args=subset_args_dl_eqf_xfmr,
    new_max_total_counts_args=None, 
    no_outg_idxs=no_outg_idxs, 
    adjust_barplot_kwargs=dict(hatch=None), 
    inplace=False
)

reason_order=mecpo_coll_drct.get_cpo_df(
    '01-05 Days', 
    mecpo_colls_w_get_args[mecpo_idx_for_ordering]['cpo_df_name'], 
    cpo_df_subset_by_mjr_mnr_cause_args=mecpo_colls_w_get_args[mecpo_idx_for_ordering]['cpo_df_subset_by_mjr_mnr_cause_args']
).mean().sort_values(ascending=False).index.tolist()

#-----
fig,axs = draw_cpo_dfs_full_vs_direct_3x2_v2(
    fig_num=1000, 
    mecpo_colls_w_get_args=mecpo_colls_w_get_args, 
    reason_order=reason_order, 
    n_reason_to_include=10, 
    is_rcpo=False, 
    replace_xtick_labels_with_ints=False, 
    include_xticklabels_for_all=False, 
    suptitle='DL_EQF_XFMR', 
    row_major=False, 
    save_args=dict(
        save_fig=save_fig,
        save_dir=save_dir, 
        save_name=save_name
    ), 
    sharex=True,
    sharey=True
)

In [ ]:
common_barplot_kwargs = dict(alpha=0.25, fill=True, edgecolor='black')
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
reason_order = mecpo_coll_drct.get_cpo_df('01-05 Days', 'rcpo_df_norm').mean().sort_values(ascending=False).index.tolist()
n_reason_to_include=10
ax0 = Plot_Bar.plot_multiple_barplots(
    ax=ax0, 
    dfs_w_args=[
        (mecpo_coll_full.get_cpo_df('01-05 Days', 'rcpo_df_norm_by_xfmr_nSNs'), {**dict(facecolor='red', hatch=None, label="Outages (All Xfmrs) - Outage nSNs Norm"), **common_barplot_kwargs}), 
        (mecpo_coll_full.get_cpo_df(
            '01-05 Days', 
            'rcpo_df_norm_by_xfmr_nSNs', 
            cpo_df_subset_by_mjr_mnr_cause_args=dict(
                mjr_cause='DL',
                mnr_cause='OL', 
                addtnl_slicers=None
            )
        ), 
         {**dict(facecolor='red', hatch='//', label="DL_OL"), **common_barplot_kwargs}), 
        (mecpo_coll_full.get_cpo_df(
            '01-05 Days', 
            'rcpo_df_norm_by_xfmr_nSNs', 
            cpo_df_subset_by_mjr_mnr_cause_args=dict(
                mjr_cause='DL',
                mnr_cause='EQF', 
                addtnl_slicers=None
            )
        ), 
         {**dict(facecolor='red', hatch='-', label="DL_EQF"), **common_barplot_kwargs}), 
        (mecpo_coll_full.get_cpo_df(
            '01-05 Days', 
            'rcpo_df_norm_by_xfmr_nSNs', 
            cpo_df_subset_by_mjr_mnr_cause_args=dict(
                mjr_cause='DL',
                mnr_cause='EQF', 
                addtnl_slicers=[dict(column='EQUIP_TYP_NM', value=['TRANSFORMER, OH', 'TRANSFORMER, UG'])] 
            )
        ), 
         {**dict(facecolor='red', hatch='\\\\', label="DL_EQF_XFMR"), **common_barplot_kwargs}), 
        
        (mecpo_coll_drct.get_cpo_df('01-05 Days', 'rcpo_df_norm_by_xfmr_nSNs'), {**dict(facecolor='green', hatch=None, label="Outages (Xfmr tied to Outg. Location ID) - Direct Outage nSNs Norm"), **common_barplot_kwargs}),
        (mecpo_coll_drct.get_cpo_df(
            '01-05 Days', 
            'rcpo_df_norm_by_xfmr_nSNs', 
            cpo_df_subset_by_mjr_mnr_cause_args=dict(
                mjr_cause='DL',
                mnr_cause='OL', 
                addtnl_slicers=None
            )
        ), 
         {**dict(facecolor='green', hatch='//', label="DL_OL"), **common_barplot_kwargs}),
        (mecpo_coll_drct.get_cpo_df(
            '01-05 Days', 
            'rcpo_df_norm_by_xfmr_nSNs', 
            cpo_df_subset_by_mjr_mnr_cause_args=dict(
                mjr_cause='DL',
                mnr_cause='EQF', 
                addtnl_slicers=None
            )
        ), 
         {**dict(facecolor='green', hatch='-', label="DL_EQF"), **common_barplot_kwargs}),
        (mecpo_coll_drct.get_cpo_df(
            '01-05 Days', 
            'rcpo_df_norm_by_xfmr_nSNs', 
            cpo_df_subset_by_mjr_mnr_cause_args=dict(
                mjr_cause='DL',
                mnr_cause='EQF', 
                addtnl_slicers=[dict(column='EQUIP_TYP_NM', value=['TRANSFORMER, OH', 'TRANSFORMER, UG'])] 
            )
        ), 
         {**dict(facecolor='green', hatch='\\\\', label="DL_EQF_XFMR"), **common_barplot_kwargs}),
        #(mecpo_coll_no_outg.get_cpo_df('01-05 Days', 'rcpo_df_norm'), {**dict(facecolor='orange', hatch=None, label="No Outages"), **common_barplot_kwargs})
    ], 
    order=reason_order, 
    n_bars_to_include=n_reason_to_include, 
    keep_edges_opaque=True, 
    include_hatches=False, 
    draw_side_by_side=True, 
    draw_single_idx_full_width=None,     
    run_set_general_plotting_args=True, 
    draw_legend=True, 
    legend_args=dict(fontsize=15), 
    title_args=dict(label=f'End Events', fontsize=20), 
    ylabel_args = dict(ylabel=f'Normalized Counts', fontsize=20, x=0.0, y=0.4, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='Reason', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=90, labelsize=15), 
              dict(axis='y', labelsize=15)]
)

In [ ]:
mecpo_colls_w_get_args_all_outgs = [
    mecpo_coll_w_get_args_full, 
    mecpo_coll_w_get_args_drct, 
    mecpo_coll_w_get_args_no_outg
]
mecpo_colls_w_get_args_all_outgs=adjust_all_subset_by_mjr_mnr_cause_args(
    mecpo_colls_w_get_args=mecpo_colls_w_get_args_all_outgs, 
    new_cpo_df_subset_by_mjr_mnr_cause_args=subset_args_all_outgs,
    new_max_total_counts_args=None, 
    adjust_barplot_kwargs=dict(hatch=None), 
    inplace=False
)
#----------
mecpo_colls_w_get_args_dl_ol = [
    mecpo_coll_w_get_args_full, 
    mecpo_coll_w_get_args_drct
]
mecpo_colls_w_get_args_dl_ol=adjust_all_subset_by_mjr_mnr_cause_args(
    mecpo_colls_w_get_args=mecpo_colls_w_get_args_dl_ol, 
    new_cpo_df_subset_by_mjr_mnr_cause_args=subset_args_dl_ol,
    new_max_total_counts_args=None, 
    adjust_barplot_kwargs=dict(hatch='//', label='DL_OL'), 
    inplace=False
)
#----------
mecpo_colls_w_get_args_dl_eqf = adjust_all_subset_by_mjr_mnr_cause_args(
    mecpo_colls_w_get_args=mecpo_colls_w_get_args_dl_ol, 
    new_cpo_df_subset_by_mjr_mnr_cause_args=subset_args_dl_eqf,
    new_max_total_counts_args=None, 
    adjust_barplot_kwargs=dict(hatch='-', label='DL_EQF'), 
    inplace=False
)
#----------
mecpo_colls_w_get_args_dl_eqf_xfmr = adjust_all_subset_by_mjr_mnr_cause_args(
    mecpo_colls_w_get_args=mecpo_colls_w_get_args_dl_ol, 
    new_cpo_df_subset_by_mjr_mnr_cause_args=subset_args_dl_eqf_xfmr,
    new_max_total_counts_args=None, 
    adjust_barplot_kwargs=dict(hatch='\\\\', label='DL_EQF_XFMR'), 
    inplace=False
)



In [ ]:
mecpo_colls_w_get_args_FULL = [
    *mecpo_colls_w_get_args_all_outgs, 
    *mecpo_colls_w_get_args_dl_ol, 
    *mecpo_colls_w_get_args_dl_eqf, 
    *mecpo_colls_w_get_args_dl_eqf_xfmr
]

In [ ]:
# mecpo_colls_w_get_args_FULL

In [ ]:
save_name = f'RCPO_dfs_3x2_ALL.{save_ext}'
#-----
fig,axs = draw_cpo_dfs_full_vs_direct_3x2_v2(
    fig_num=1000, 
    mecpo_colls_w_get_args=mecpo_colls_w_get_args_FULL, 
    reason_order=mecpo_coll_drct.get_cpo_df('01-05 Days', 'rcpo_df_norm_by_xfmr_nSNs').mean().sort_values(ascending=False).index.tolist(), 
    n_reason_to_include=10, 
    is_rcpo=True, 
    replace_xtick_labels_with_ints=True, 
    suptitle='Full', 
    row_major=False, 
    save_args=dict(
        save_fig=save_fig,
        save_dir=save_dir, 
        save_name=save_name
    ), 
    sharex=True,
    sharey=True
)

In [ ]:
unq_colors = []
subset_cause_args = []
for coll_w_get_args in mecpo_colls_w_get_args_FULL:
    unq_colors.append(coll_w_get_args['mecpo_coll'].barplot_kwargs_shared['facecolor'])
    subset_cause_args.append(coll_w_get_args['cpo_df_subset_by_mjr_mnr_cause_args'])
unq_colors = list(set(unq_colors))
#-----
unq_subset_cause_args = []
for subset in subset_cause_args:
    if subset in unq_subset_cause_args:
        continue
    unq_subset_cause_args.append(subset)

In [ ]:
unq_subset_cause_args

In [ ]:
unq_colors

In [ ]:
mecpo_colls_w_get_args_FULL_sorted = []
for color in unq_colors:
    for subset in unq_subset_cause_args:
        for coll_w_get_args in mecpo_colls_w_get_args_FULL:
            if(coll_w_get_args['mecpo_coll'].barplot_kwargs_shared['facecolor']==color and 
               coll_w_get_args['cpo_df_subset_by_mjr_mnr_cause_args']==subset):
                mecpo_colls_w_get_args_FULL_sorted.append(coll_w_get_args)
assert(len(mecpo_colls_w_get_args_FULL_sorted)==len(mecpo_colls_w_get_args_FULL))

In [ ]:
save_name = f'RCPO_dfs_3x2_ALL_SRTD.{save_ext}'
#-----
fig,axs = draw_cpo_dfs_full_vs_direct_3x2_v2(
    fig_num=1000, 
    mecpo_colls_w_get_args=mecpo_colls_w_get_args_FULL_sorted, 
    reason_order=mecpo_coll_drct.get_cpo_df('01-05 Days', 'rcpo_df_norm_by_xfmr_nSNs').mean().sort_values(ascending=False).index.tolist(), 
    n_reason_to_include=10, 
    is_rcpo=True, 
    replace_xtick_labels_with_ints=True, 
    suptitle='Full', 
    row_major=False, 
    save_args=dict(
        save_fig=save_fig,
        save_dir=save_dir, 
        save_name=save_name
    ), 
    sharex=True,
    sharey=True
)

# TOTAL COUNTS

In [ ]:
cpo_df_subset_by_mjr_mnr_cause_args=None
#-------------------------
total_event_counts_full = mecpo_coll_full.get_total_event_counts(
    '01-05 Days', 
    'rcpo_df_norm_by_outg_nSNs', 
    cpo_df_subset_by_mjr_mnr_cause_args=cpo_df_subset_by_mjr_mnr_cause_args)

total_event_counts_drct = mecpo_coll_drct.get_total_event_counts(
    '01-05 Days', 
    'rcpo_df_norm_by_prim_nSNs', 
    cpo_df_subset_by_mjr_mnr_cause_args=cpo_df_subset_by_mjr_mnr_cause_args)

total_event_counts_no_outg = mecpo_coll_no_outg.get_total_event_counts(
    '01-05 Days', 
    'rcpo_df_norm_by_xfmr_nSNs', 
    cpo_df_subset_by_mjr_mnr_cause_args=cpo_df_subset_by_mjr_mnr_cause_args)

In [ ]:
bin_size = 10
min_max_and_bin_size = (
    0, 
    np.ceil(total_event_counts_full['total_counts'].max()), 
    bin_size
)

fig, ax0 = Plot_General.default_subplots(1,1)
Plot_Hist.plot_hist(ax=ax0, df=total_event_counts_full, x_col='total_counts', min_max_and_bin_size=min_max_and_bin_size)
ax0.set_yscale('log')

In [ ]:
bin_size = 10
min_max_and_bin_size = (
    0, 
    np.ceil(total_event_counts_drct['total_counts'].max()), 
    bin_size
)

fig, ax0 = Plot_General.default_subplots(1,1)
Plot_Hist.plot_hist(ax=ax0, df=total_event_counts_drct, x_col='total_counts', min_max_and_bin_size=min_max_and_bin_size)
ax0.set_yscale('log')

In [ ]:
bin_size = 10
min_max_and_bin_size = (
    0, 
    np.ceil(total_event_counts_no_outg['total_counts'].max()), 
    bin_size
)

fig, ax0 = Plot_General.default_subplots(1,1)
Plot_Hist.plot_hist(ax=ax0, df=total_event_counts_no_outg, x_col='total_counts', min_max_and_bin_size=min_max_and_bin_size)
ax0.set_yscale('log')

In [ ]:
bin_size = 10
min_max_and_bin_size = (
    0, 
    np.ceil(total_event_counts_no_outg['total_counts'].max()), 
    bin_size
)

fig, ax0 = Plot_General.default_subplots(1,1)
Plot_Hist.plot_multiple_hists(
    ax=ax0, 
    dfs_w_args=[
        (total_event_counts_full, mecpo_coll_full.barplot_kwargs_shared), 
        (total_event_counts_drct, mecpo_coll_drct.barplot_kwargs_shared), 
        (total_event_counts_no_outg, mecpo_coll_no_outg.barplot_kwargs_shared)
    ], 
    x_col='total_counts', 
    min_max_and_bin_size=min_max_and_bin_size, 
    draw_side_by_side=True, 
    draw_legend=True
)
ax0.set_yscale('log')

In [ ]:
bin_size = 10
min_max_and_bin_size = (
    0, 
    np.ceil(total_event_counts_no_outg['total_counts'].max()), 
    bin_size
)

fig, ax0 = Plot_General.default_subplots(1,1)
Plot_Hist.plot_multiple_hists(
    ax=ax0, 
    dfs_w_args=[
        (total_event_counts_full, mecpo_coll_full.barplot_kwargs_shared), 
        (total_event_counts_drct, mecpo_coll_drct.barplot_kwargs_shared), 
        (total_event_counts_no_outg, mecpo_coll_no_outg.barplot_kwargs_shared)
    ], 
    x_col='total_counts', 
    min_max_and_bin_size=min_max_and_bin_size, 
    stat='density', 
    draw_side_by_side=True, 
    draw_legend=True
)
ax0.set_yscale('log')

In [ ]:
bin_size = 10
min_max_and_bin_size = (
    0, 
    np.ceil(total_event_counts_no_outg['total_counts'].max()), 
    bin_size
)

fig, ax0 = Plot_General.default_subplots(1,1)
Plot_Hist.plot_multiple_hists(
    ax=ax0, 
    dfs_w_args=[
        (total_event_counts_full, mecpo_coll_full.barplot_kwargs_shared), 
        (total_event_counts_drct, mecpo_coll_drct.barplot_kwargs_shared), 
        (total_event_counts_no_outg, mecpo_coll_no_outg.barplot_kwargs_shared)
    ], 
    x_col='total_counts', 
    min_max_and_bin_size=min_max_and_bin_size, 
    draw_side_by_side=True, 
    draw_legend=True
)

In [ ]:
assert(0)

# DEV

In [ ]:
dev_df_full = mecpo_coll_full.get_cpo_df('01-05 Days', 'rcpo_df_norm_by_outg_nSNs')
dev_df_drct = mecpo_coll_drct.get_cpo_df('01-05 Days', 'rcpo_df_norm_by_prim_nSNs')
dev_df_no_outg = mecpo_coll_no_outg.get_cpo_df('01-05 Days', 'rcpo_df_norm_by_xfmr_nSNs')

reason_order_df_name = 'rcpo_df_norm_by_prim_nSNs'
reason_order=mecpo_coll_drct.get_cpo_df(
    '01-05 Days', 
    reason_order_df_name, 
    cpo_df_subset_by_mjr_mnr_cause_args=None
).mean().sort_values(ascending=False).index.tolist()
#---------------
xtick_elements = reason_order
xtick_rename_dict = {xtick_el:i+1 for i,xtick_el in enumerate(xtick_elements)}
xticks = np.arange(0, len(xtick_rename_dict), 10)

In [ ]:
print(dev_df_full.shape)
print(dev_df_drct.shape)
print(dev_df_no_outg.shape)

In [ ]:
# dev_df_full    = dev_df_full[reason_order].iloc[:10]
# dev_df_drct    = dev_df_drct[reason_order].iloc[:10]
# dev_df_no_outg = dev_df_no_outg[reason_order].iloc[:10]

dev_df_full    = dev_df_full[reason_order]
dev_df_drct    = dev_df_drct[reason_order]
dev_df_no_outg = dev_df_no_outg[reason_order]

In [ ]:
fig, axs = Plot_General.default_subplots(n_x=1, n_y=3, fig_num=1)
ylim = None
#-------------------------
dev_df_full.T.plot.line(ax=axs[0], alpha=0.1, color='red', linewidth=1)
Plot_General.set_general_plotting_args(
    axs[0], 
    draw_legend=False, 
    ax_args = dict(
        ylim=ylim, 
        xticks=xticks,
        xticklabels=np.array(list(xtick_rename_dict.values()))[xticks]
    ), 
    ylabel_args = dict(ylabel=f'Normalized Counts', fontsize=20, x=0.0, y=0.4, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=90, labelsize=10), 
               dict(axis='y', labelsize=15)]
)
#-------------------------
dev_df_drct.T.plot.line(ax=axs[1], alpha=0.1, color='green')
Plot_General.set_general_plotting_args(
    axs[1], 
    draw_legend=False, 
    ax_args = dict(
        ylim=ylim, 
        xticks=xticks,
        xticklabels=np.array(list(xtick_rename_dict.values()))[xticks]
    ), 
    ylabel_args = dict(ylabel=f'Normalized Counts', fontsize=20, x=0.0, y=0.4, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=90, labelsize=10), 
               dict(axis='y', labelsize=15)]
)
#-------------------------
dev_df_no_outg.T.plot.line(ax=axs[2], alpha=0.1, color='orange')
Plot_General.set_general_plotting_args(
    axs[2], 
    draw_legend=False, 
    ax_args = dict(
        ylim=ylim, 
        xticks=xticks,
        xticklabels=np.array(list(xtick_rename_dict.values()))[xticks]
    ), 
    ylabel_args = dict(ylabel=f'Normalized Counts', fontsize=20, x=0.0, y=0.4, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=90, labelsize=10), 
               dict(axis='y', labelsize=15)]
)
#-------------------------

In [ ]:
fig, axs = Plot_General.default_subplots(n_x=1, n_y=3, fig_num=1)
ylim = (0,25)
#-------------------------
dev_df_full.T.plot.line(ax=axs[0], alpha=0.1, color='red', linewidth=1)
Plot_General.set_general_plotting_args(
    axs[0], 
    draw_legend=False, 
    ax_args = dict(
        ylim=ylim, 
        xticks=xticks,
        xticklabels=np.array(list(xtick_rename_dict.values()))[xticks]
    ), 
    ylabel_args = dict(ylabel=f'Normalized Counts', fontsize=20, x=0.0, y=0.4, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=90, labelsize=10), 
               dict(axis='y', labelsize=15)]
)
#-------------------------
dev_df_drct.T.plot.line(ax=axs[1], alpha=0.1, color='green')
Plot_General.set_general_plotting_args(
    axs[1], 
    draw_legend=False, 
    ax_args = dict(
        ylim=ylim, 
        xticks=xticks,
        xticklabels=np.array(list(xtick_rename_dict.values()))[xticks]
    ), 
    ylabel_args = dict(ylabel=f'Normalized Counts', fontsize=20, x=0.0, y=0.4, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=90, labelsize=10), 
               dict(axis='y', labelsize=15)]
)
#-------------------------
dev_df_no_outg.T.plot.line(ax=axs[2], alpha=0.1, color='orange')
Plot_General.set_general_plotting_args(
    axs[2], 
    draw_legend=False, 
    ax_args = dict(
        ylim=ylim, 
        xticks=xticks,
        xticklabels=np.array(list(xtick_rename_dict.values()))[xticks]
    ), 
    ylabel_args = dict(ylabel=f'Normalized Counts', fontsize=20, x=0.0, y=0.4, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=90, labelsize=10), 
               dict(axis='y', labelsize=15)]
)
#-------------------------
subplot_layout_params = Plot_General.get_subplot_layout_params(fig)
Plot_General.generate_xtick_labels_legend_textbox(
    fig=fig, 
    xtick_rename_dict=xtick_rename_dict, 
    text_x_pos=1.02*subplot_layout_params['right'], 
    text_y_pos=subplot_layout_params['top'], 
    fontsize=8, 
    n_cols=3, 
    col_padding = 0.01
)

In [ ]:
fig, ax = Plot_General.default_subplots(n_x=1, n_y=1, fig_num=1)
dev_df_no_outg.T.plot.line(ax=ax, alpha=0.1, color='orange')
Plot_General.set_general_plotting_args(
    ax, 
    draw_legend=False, 
    ax_args = dict(
        ylim=(0,25), 
        xticks = xticks,
        xticklabels=np.array(list(xtick_rename_dict.values()))[xticks]
    ), 
    ylabel_args = dict(ylabel=f'Normalized Counts', fontsize=20, x=0.0, y=0.4, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=0, labelsize=10), 
               dict(axis='y', labelsize=15)]
)

subplot_layout_params = Plot_General.get_subplot_layout_params(fig)
Plot_General.generate_xtick_labels_legend_textbox(
    fig=fig, 
    xtick_rename_dict=xtick_rename_dict, 
    text_x_pos=1.02*subplot_layout_params['right'], 
    text_y_pos=subplot_layout_params['top'], 
    n_cols=1, 
    col_padding = 0.01
)

In [ ]:
fig, ax = Plot_General.default_subplots(n_x=1, n_y=1, fig_num=1)
dev_df_no_outg.T.iloc[:,0].plot.line(ax=ax, alpha=0.1, color='orange')

In [ ]:
# %matplotlib auto
fig, ax = Plot_General.default_subplots(n_x=1, n_y=1, fig_num=1)
dev_df_no_outg.iloc[0].plot.line(ax=ax, alpha=0.1, color='orange')

In [ ]:
# importing all necessary libraries
from itertools import count
import random
import matplotlib
from matplotlib import animation
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
# %matplotlib qt
# Enable interactive plot
# %matplotlib notebook

import ffmpeg
from IPython.display import HTML

In [ ]:
random.randint(0,1000)

In [ ]:
def init_f():
    fig, ax = Plot_General.default_subplots(n_x=1, n_y=1, fig_num=random.randint(0,1000))
    Plot_General.set_general_plotting_args(
        ax, 
        draw_legend=False, 
        ax_args = dict(
            ylim=(0,25), 
            xticks = xticks,
            xticklabels=np.array(list(xtick_rename_dict.values()))[xticks]
        ), 
        ylabel_args = dict(ylabel=f'Normalized Counts', fontsize=20, x=0.0, y=0.4, ha='left', va='bottom'), 
        xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
        tick_args=[dict(axis='x', labelrotation=0, labelsize=10), 
                   dict(axis='y', labelsize=15)]
    )    

In [ ]:
fig, ax = Plot_General.default_subplots(n_x=1, n_y=1, fig_num=1)
dev_df_no_outg.iloc[0].plot.line(ax=ax, alpha=0.1, color='orange')
ax=Plot_General.set_general_plotting_args(
    ax, 
    draw_legend=False, 
    ax_args = dict(
        ylim=(0,1), 
        xticks = xticks,
        xticklabels=np.array(list(xtick_rename_dict.values()))[xticks]
    ), 
    ylabel_args = dict(ylabel=f'Normalized Counts', fontsize=20, x=0.0, y=0.4, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=0, labelsize=10), 
               dict(axis='y', labelsize=15)]
)

def animate(i):
    for artist in ax.lines + plt.gca().collections:
        artist.remove()
    dev_df_no_outg.iloc[i].plot.line(ax=ax, alpha=1.0, color='orange')
    
anim = FuncAnimation(fig, animate, interval=30, frames=100)
# dev_df_no_outg.iloc[0].plot.line(ax=ax, alpha=0.1, color='orange')

In [ ]:
HTML(anim.to_jshtml())

In [ ]:
colors = Plot_General.get_standard_colors(10)

In [ ]:
save_dir = r'C:\Users\s346557\Downloads\TMP'

In [ ]:
for i in range(dev_df_no_outg.shape[0]):
    fig, ax = Plot_General.default_subplots(n_x=1, n_y=1, fig_num=i)
    # dev_df_no_outg.T.plot.line(ax=ax, alpha=0.1, color='orange')
    dev_df_no_outg.iloc[i].plot.line(ax=ax, alpha=1.0, color=colors[i%10])
    Plot_General.set_general_plotting_args(
        ax, 
        draw_legend=False, 
        ax_args = dict(
            ylim=(0,1), 
            xticks = xticks,
            xticklabels=np.array(list(xtick_rename_dict.values()))[xticks]
        ), 
        ylabel_args = dict(ylabel=f'Normalized Counts', fontsize=20, x=0.0, y=0.4, ha='left', va='bottom'), 
        xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
        tick_args=[dict(axis='x', labelrotation=0, labelsize=10), 
                   dict(axis='y', labelsize=15)]
    )

    subplot_layout_params = Plot_General.get_subplot_layout_params(fig)
    Plot_General.generate_xtick_labels_legend_textbox(
        fig=fig, 
        xtick_rename_dict=xtick_rename_dict, 
        text_x_pos=1.02*subplot_layout_params['right'], 
        text_y_pos=subplot_layout_params['top'], 
        n_cols=1, 
        col_padding = 0.01
    )
    Plot_General.save_fig(fig, save_dir, f'line_{i}.png')
    plt.close(fig)

In [ ]:
# batch_idxs = Utilities.get_batch_idx_pairs(dev_df_no_outg.shape[0], 10)
# for i,batch_idx_i in enumerate(batch_idxs):
#     fig, ax = Plot_General.default_subplots(n_x=1, n_y=1, fig_num=i)
#     # dev_df_no_outg.T.plot.line(ax=ax, alpha=0.1, color='orange')
#     dev_df_no_outg.T.iloc[:, batch_idx_i[0]:batch_idx_i[1]].plot.line(ax=ax, alpha=1.0, color=colors)
#     Plot_General.set_general_plotting_args(
#         ax, 
#         draw_legend=False, 
#         ax_args = dict(
#             ylim=(0,1), 
#             xticks = xticks,
#             xticklabels=np.array(list(xtick_rename_dict.values()))[xticks]
#         ), 
#         ylabel_args = dict(ylabel=f'Normalized Counts', fontsize=20, x=0.0, y=0.4, ha='left', va='bottom'), 
#         xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
#         tick_args=[dict(axis='x', labelrotation=0, labelsize=10), 
#                    dict(axis='y', labelsize=15)]
#     )

#     subplot_layout_params = Plot_General.get_subplot_layout_params(fig)
#     Plot_General.generate_xtick_labels_legend_textbox(
#         fig=fig, 
#         xtick_rename_dict=xtick_rename_dict, 
#         text_x_pos=1.02*subplot_layout_params['right'], 
#         text_y_pos=subplot_layout_params['top'], 
#         n_cols=1, 
#         col_padding = 0.01
#     )
#     Plot_General.save_fig(fig, save_dir, f'line_{i}.png')
#     plt.close(fig)

In [ ]:
import imageio

In [ ]:
# with imageio.get_writer('line.gif', mode='i') as writer:
#     for i in range(2, SIZE+1):
#         image = imageio.imread(f'line-{i}.png')
#         writer.append_data(image)

In [ ]:
img_paths = Utilities.find_all_paths(
    base_dir=save_dir, 
    glob_pattern=r'line_*.png'
)

In [ ]:
with imageio.get_writer(os.path.join(save_dir, 'lines.gif'), mode='i') as writer:
    for img_path in img_paths:
        image = imageio.imread(img_path)
        writer.append_data(image)

In [ ]:
mecpo_coll_full.cpo_df_names

# Taken from original version

In [ ]:
rcpo_df_norm = mecpo_coll_full.get_cpo_df('01-05 Days', 'rcpo_df_norm').copy()
rcpo_df_norm_by_xfmr_nSNs = mecpo_coll_full.get_cpo_df('01-05 Days', 'rcpo_df_norm_by_xfmr_nSNs').copy()

rcpo_prim_strict_df_norm = mecpo_coll_drct.get_cpo_df('01-05 Days', 'rcpo_df_norm').copy()
rcpo_prim_strict_df_norm_by_xfmr_nSNs = mecpo_coll_drct.get_cpo_df('01-05 Days', 'rcpo_df_norm_by_xfmr_nSNs').copy()

rcpo_no_outg_df_norm = mecpo_coll_no_outg.get_cpo_df('01-05 Days', 'rcpo_df_norm').copy()


In [ ]:
rcpo_df_norm.columns

In [ ]:
rcpo_df_norm['Under Voltage for meter'].describe()

In [ ]:
rcpo_df_norm.describe()

In [ ]:
rcpo_df_norm[rcpo_df_norm.mean().sort_values(ascending=False).index.tolist()].describe()

In [ ]:
rcpo_df_norm[rcpo_df_norm.mean().sort_values(ascending=False).index.tolist()].describe().T

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
order=rcpo_df_norm.mean().sort_values(ascending=False).index.tolist()
rcpo_df_norm[order].describe().T.drop('count', axis=1).plot(ax=ax0)
ax0.set_xticks(range(len(order)))
ax0.set_xticklabels(order)
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)

In [ ]:
idk = rcpo_df_norm[rcpo_df_norm.mean().sort_values(ascending=False).index.tolist()].describe().T

In [ ]:
idk

In [ ]:
idk35 = rcpo_df_norm[rcpo_df_norm.mean().sort_values(ascending=False).index.tolist()].quantile(0.35)

In [ ]:
rcpo_df_norm[rcpo_df_norm.mean().sort_values(ascending=False).index.tolist()].quantile(0.35)

In [ ]:
idk35.name='35%'

In [ ]:
idk.merge(idk35, left_index=True, right_index=True)

In [ ]:
quantiles = np.arange(0.1,1.0,0.1).tolist()
quantiles.extend([0.95,0.975,0.99,0.999])

In [ ]:
quant_series = []
for quant in quantiles:
    q_i = rcpo_df_norm[rcpo_df_norm.mean().sort_values(ascending=False).index.tolist()].quantile(quant)
    q_i.name = f'{round(quant*100,1)}%'
    quant_series.append(q_i)

In [ ]:
quant_df = pd.concat(quant_series, axis=1)

In [ ]:
quant_df

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
order=rcpo_df_norm.mean().sort_values(ascending=False).index.tolist()
quant_df.loc[order].plot(ax=ax0)
ax0.set_xticks(range(len(order)))
ax0.set_xticklabels(order)
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
order=rcpo_df_norm.mean().sort_values(ascending=False).index.tolist()[:10]
quant_df.loc[order].plot(ax=ax0)
ax0.set_xticks(range(len(order)))
ax0.set_xticklabels(order)
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)

In [ ]:
pd.melt(rcpo_df_norm)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
sns.boxplot(ax=ax0, x="variable", y="value", data=pd.melt(rcpo_df_norm), 
            order=rcpo_df_norm.mean().sort_values(ascending=False).index.tolist())
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
sns.boxplot(ax=ax0, x="variable", y="value", data=pd.melt(rcpo_df_norm), 
            order=rcpo_df_norm.mean().sort_values(ascending=False).index.tolist())
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)
ax0.set_yscale('log')

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
sns.boxplot(ax=ax0, x="variable", y="value", data=pd.melt(rcpo_df_norm), 
            order=rcpo_df_norm.mean().sort_values(ascending=False).index.tolist())
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)
ax0.set_ylim(ymin=0.0, ymax=15.0)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
sns.boxplot(ax=ax0, x="variable", y="value", data=pd.melt(rcpo_df_norm), showfliers=False, 
            order=rcpo_df_norm.mean().sort_values(ascending=False).index.tolist())
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
sns.barplot(ax=ax0, data=rcpo_df_norm)
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
sns.barplot(ax=ax0, data=rcpo_df_norm, ci='sd')
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
sns.barplot(ax=ax0, data=rcpo_df_norm, order=rcpo_df_norm.mean().sort_values(ascending=False).index.tolist())
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
sns.barplot(ax=ax0, data=rcpo_prim_strict_df_norm, order=rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist())
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[6, 14])
sns.barplot(
    ax=ax0, 
    data=rcpo_prim_strict_df_norm, 
    order=rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist(), 
    orient='h')
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
sns.barplot(ax=ax0, data=rcpo_df_norm, order=rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist())
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
div_drawn_width_by=2
#-------------------------
n_patches_beg = len(ax0.patches)
sns.barplot(
    ax=ax0, 
    data=rcpo_df_norm, 
    order=rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist(), 
    color='red', 
    alpha=0.25, 
    label="Outages (All Xfmrs)"
)
n_patches_end = len(ax0.patches)
relative_position_idx=0
ax0 = Plot_Bar.adjust_bar_and_line_positions_and_widths(ax0, div_drawn_width_by, relative_position_idx, 
                                                        i_patch_beg=n_patches_beg, i_patch_end=n_patches_end)
#-------------------------
n_patches_beg = len(ax0.patches)
sns.barplot(
    ax=ax0, 
    data=rcpo_prim_strict_df_norm, 
    order=rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist(), 
    color='green', 
    alpha=0.25, 
    label="Outages (Xfmr tied to Outg. Location ID)"
)
n_patches_end = len(ax0.patches)
relative_position_idx=1
ax0 = Plot_Bar.adjust_bar_and_line_positions_and_widths(ax0, div_drawn_width_by, relative_position_idx, 
                                                        i_patch_beg=n_patches_beg, i_patch_end=n_patches_end)
#-------------------------
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)
ax0.legend()

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
div_drawn_width_by=2
reason_order = rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist()
reason_order = [x for x in reason_order if x!= '_nSNs']
n_reason_to_include=15
df_to_plot             = rcpo_df_norm[reason_order]
df_to_plot_prim_strict = rcpo_prim_strict_df_norm[reason_order]
if n_reason_to_include is not None:
    df_to_plot             = df_to_plot.iloc[:, :n_reason_to_include]
    df_to_plot_prim_strict = df_to_plot_prim_strict.iloc[:, :n_reason_to_include]
    reason_order = df_to_plot_prim_strict.mean().sort_values(ascending=False).index.tolist()
#-------------------------
n_patches_beg = len(ax0.patches)
sns.barplot(
    ax=ax0, 
    data=df_to_plot, 
    order=reason_order, 
    color='red', 
    alpha=0.25, 
    label="Outages (All Xfmrs)"
)
n_patches_end = len(ax0.patches)
relative_position_idx=0
ax0 = Plot_Bar.adjust_bar_and_line_positions_and_widths(ax0, div_drawn_width_by, relative_position_idx, 
                                                        i_patch_beg=n_patches_beg, i_patch_end=n_patches_end)
#-------------------------
n_patches_beg = len(ax0.patches)
sns.barplot(
    ax=ax0, 
    data=df_to_plot_prim_strict, 
    order=reason_order, 
    color='green', 
    alpha=0.25, 
    label="Outages (Xfmr tied to Outg. Location ID)"
)
n_patches_end = len(ax0.patches)
relative_position_idx=1
ax0 = Plot_Bar.adjust_bar_and_line_positions_and_widths(ax0, div_drawn_width_by, relative_position_idx, 
                                                        i_patch_beg=n_patches_beg, i_patch_end=n_patches_end)
#-------------------------
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)
ax0.legend();

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
div_drawn_width_by=2
reason_order = rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist()
reason_order = [x for x in reason_order if x!= '_nSNs']
n_reason_to_include=15
#-------------------------
relative_position_idx=0
Plot_Bar.plot_barplot(
    ax=ax0, 
    df=rcpo_df_norm, 
    order=reason_order, 
    n_bars_to_include=n_reason_to_include, 

    barplot_kwargs = dict(alpha=0.25, fill=True, facecolor='red', edgecolor='black', hatch='//', label="Outages (All Xfmrs)"), 
    keep_edges_opaque=True, 
    div_drawn_width_by=div_drawn_width_by, 
    relative_position_idx=relative_position_idx, 
    run_set_general_plotting_args=False, 
    draw_legend=True, 
    legend_args=dict(fontsize=15), 
    title_args=dict(label=f'Title', fontsize=20), 
    ylabel_args = dict(ylabel=f'y-label', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='x-label', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=90, labelsize=7), 
              dict(axis='y', labelsize=15)]
)
#-------------------------
relative_position_idx=1
Plot_Bar.plot_barplot(
    ax=ax0, 
    df=rcpo_prim_strict_df_norm, 
    order=reason_order, 
    n_bars_to_include=n_reason_to_include, 

    barplot_kwargs = dict(alpha=0.25, fill=True, facecolor='green', edgecolor='black', hatch='//', label="Outages (Xfmr tied to Outg. Location ID)"), 
    keep_edges_opaque=True, 
    div_drawn_width_by=div_drawn_width_by, 
    relative_position_idx=relative_position_idx, 
    run_set_general_plotting_args=True, 
    draw_legend=True, 
    legend_args=dict(fontsize=15), 
    title_args=dict(label=f'Title', fontsize=20), 
    ylabel_args = dict(ylabel=f'y-label', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='x-label', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=90, labelsize=12), 
              dict(axis='y', labelsize=15)], 
    grid_args=[dict(visible=False, axis='x', which='both', color='red'), 
               dict(visible=False, axis='y', which='major', color='green')]
)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
reason_order = rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist()
n_reason_to_include=15
ax0 = Plot_Bar.plot_multiple_barplots(
    ax=ax0, 
    dfs_w_args=[rcpo_df_norm, rcpo_prim_strict_df_norm], 
    order=reason_order, 
    n_bars_to_include=n_reason_to_include, 
    keep_edges_opaque=True, 
    include_hatches=False, 
    draw_side_by_side=True, 
    draw_single_idx_full_width=None,     
    run_set_general_plotting_args=True, 
    draw_legend=False, 
    legend_args=dict(fontsize=15), 
    title_args=dict(label=f'Title', fontsize=20), 
    ylabel_args = dict(ylabel=f'y-label', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='x-label', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=90, labelsize=7), 
              dict(axis='y', labelsize=15)]
)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
reason_order = rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist()
n_reason_to_include=15
ax0 = Plot_Bar.plot_multiple_barplots(
    ax=ax0, 
    dfs_w_args=[rcpo_df_norm, rcpo_prim_strict_df_norm, rcpo_prim_strict_df_norm], 
    order=reason_order, 
    n_bars_to_include=n_reason_to_include, 
    keep_edges_opaque=True, 
    include_hatches=False, 
    draw_side_by_side=True, 
    draw_single_idx_full_width=2,     
    run_set_general_plotting_args=True, 
    draw_legend=False, 
    legend_args=dict(fontsize=15), 
    title_args=dict(label=f'Title', fontsize=20), 
    ylabel_args = dict(ylabel=f'y-label', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='x-label', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=90, labelsize=7), 
              dict(axis='y', labelsize=15)]
)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
reason_order = rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist()
n_reason_to_include=15
ax0 = Plot_Bar.plot_multiple_barplots(
    ax=ax0, 
    dfs_w_args=[
        rcpo_df_norm, 
        rcpo_df_norm_by_xfmr_nSNs, 
        rcpo_prim_strict_df_norm, 
        rcpo_prim_strict_df_norm_by_xfmr_nSNs, 
        rcpo_no_outg_df_norm
    ], 
    order=reason_order, 
    n_bars_to_include=n_reason_to_include, 
    keep_edges_opaque=True, 
    include_hatches=False, 
    draw_side_by_side=True, 
    draw_single_idx_full_width=None,     
    run_set_general_plotting_args=True, 
    draw_legend=False, 
    legend_args=dict(fontsize=15), 
    title_args=dict(label=f'Title', fontsize=20), 
    ylabel_args = dict(ylabel=f'y-label', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='x-label', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=90, labelsize=7), 
              dict(axis='y', labelsize=15)]
)

In [ ]:
common_barplot_kwargs = dict(alpha=0.25, fill=True, edgecolor='black')
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
reason_order = rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist()
n_reason_to_include=15
ax0 = Plot_Bar.plot_multiple_barplots(
    ax=ax0, 
    dfs_w_args=[
        (rcpo_df_norm,              {**dict(facecolor='red', hatch='//', label="Outages (All Xfmrs) - EndEvents nSNs Norm"), **common_barplot_kwargs}), 
        (rcpo_df_norm_by_xfmr_nSNs, {**dict(facecolor='red', hatch='-', label="Outages (All Xfmrs) - Outage nSNs Norm"), **common_barplot_kwargs}), 
        (rcpo_prim_strict_df_norm,              {**dict(facecolor='green', hatch='//', label="Outages (Xfmr tied to Outg. Location ID) - EndEvents nSNs Norm"), **common_barplot_kwargs}), 
        (rcpo_prim_strict_df_norm_by_xfmr_nSNs, {**dict(facecolor='green', hatch='-', label="Outages (Xfmr tied to Outg. Location ID) - Outage nSNs Norm"), **common_barplot_kwargs}), 
        (rcpo_no_outg_df_norm, {**dict(facecolor='orange', hatch=None, label="No Outages"), **common_barplot_kwargs})
    ], 
    order=reason_order, 
    n_bars_to_include=n_reason_to_include, 
    keep_edges_opaque=True, 
    include_hatches=False, 
    draw_side_by_side=True, 
    draw_single_idx_full_width=None,     
    run_set_general_plotting_args=True, 
    draw_legend=True, 
    legend_args=dict(fontsize=15), 
    title_args=dict(label=f'End Events', fontsize=20), 
    ylabel_args = dict(ylabel=f'Normalized Counts', fontsize=20, x=0.0, y=0.4, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='Reason', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=90, labelsize=15), 
              dict(axis='y', labelsize=15)]
)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
ax0 = Plot_Bar.plot_barplot(
    ax=ax0, 
    df = rcpo_no_outg_df_norm, 
    order=rcpo_no_outg_df_norm.mean().sort_values(ascending=False).index.tolist(), 
    n_bars_to_include=10, 
    tick_args =dict(axis='x', labelrotation=90, labelsize=15.0)
)

In [ ]:
#barplot_kwargs = dict(alpha=0.5, fill=True, facecolor='red', edgecolor='black', hatch='//', palette='tab10')
barplot_kwargs = dict(alpha=0.5, fill=True, color='red', edgecolor='black', palette='tab10')

sns.set_theme(style="whitegrid")
tips = sns.load_dataset("tips")
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
n_patches_beg = len(ax0.patches)
sns.barplot(ax=ax0, x="day", y="total_bill", data=tips, **barplot_kwargs)

div_drawn_width_by=2
n_patches_end = len(ax0.patches)
relative_position_idx=1
ax0 = Plot_Bar.adjust_bar_and_line_positions_and_widths(ax0, div_drawn_width_by, relative_position_idx, 
                                                        i_patch_beg=n_patches_beg, i_patch_end=n_patches_end)
ax0.grid(True)

In [ ]:
dict(visible=True, axis='x', which='both', color='red')

In [ ]:
grid_args = False

In [ ]:
dfs_w_args = [rcpo_df_norm, rcpo_prim_strict_df_norm]
dfs_color_palette=None
include_hatches=False

In [ ]:
colors = Plot_General.get_standard_colors(len(dfs_w_args), palette=dfs_color_palette)
hatches = Plot_General.get_standard_hatches(len(dfs_w_args))
for i in range(len(dfs_w_args)):
    # First, normalize dfs_w_args[i] so it is a tuple whose first first element
    # is the pd.DataFrame and second is a dict containing the kwargs
    assert(Utilities.is_object_one_of_types(dfs_w_args[i], [pd.DataFrame, list, tuple]))
    if isinstance(dfs_w_args[i], pd.DataFrame):
        dfs_w_args[i] = (dfs_w_args[i], dict())
        
    # At this point, dfs_w_args[i] is definitely a list/tuple.  
    # Make sure length=2, first item is pd.DataFrame, and second is dict
    assert(len(dfs_w_args[i])==2 and 
           isinstance(dfs_w_args[i][0], pd.DataFrame) and 
           isinstance(dfs_w_args[i][1], dict))
    
    # Now, set all default values for plot_kwargs_i
    dfs_w_args[i][1]['hatch'] = dfs_w_args[i][1].get('hatch', hatches[i] if include_hatches else None)
    dfs_w_args[i][1]['color'] = dfs_w_args[i][1].get('color', colors[i])
    #dfs_w_args[i][1]['alpha'] = dfs_w_args[i][1].get('alpha', 1.0/len(dfs_w_args))
    dfs_w_args[i][1]['label'] = dfs_w_args[i][1].get('label', f'{i}')

In [ ]:
type(dfs_w_args[0][0])

In [ ]:
dfs_w_args[0][1]

In [ ]:
dfs_w_args[1][1]